In [1]:
import yfinance as yf
import numpy as np

from scipy import stats


from binance.client import Client as Client_
import pandas as pd

from datetime import datetime

from dateutil.relativedelta import relativedelta
from pandas_datareader import data
from dask.distributed import Client
from dask.distributed import as_completed
import openpyxl
import xlsxwriter
import pandas_datareader.data as web
import statsmodels.tsa.stattools as ts
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import pandas_market_calendars as mcal
import os
import sqlite3
import matplotlib.pyplot as plt
import dataframe_image as dfi
import seaborn as sns
import scipy.stats as stats
import pickle

import datetime as dt

import scipy

import sys

import os

path = os.getcwd()

print(path)

C:\Users\User\Documents\wiki\wiki\dev\python\Python-Stock\code\Screener\code


In [2]:

client_=Client_()

def get_historical_ohlc_data(symbol,past_days=None,interval=None):
    
    """Returns historcal klines from past for given symbol and interval
    past_days: how many days back one wants to download the data"""
    
    if not interval:
        interval='1h' # default interval 1 hour
    if not past_days:
        past_days=30  # default past days 30.

    start_str=str((pd.to_datetime('today')-pd.Timedelta(str(past_days)+' days')).date())
    
    D=pd.DataFrame(client_.get_historical_klines(symbol=symbol,start_str=start_str,interval=interval))
    D.columns=['open_time','open', 'high', 'low', 'close', 'volume', 'close_time', 'qav', 'num_trades', 'taker_base_vol', 'taker_quote_vol','is_best_match']
    D['open_date_time']=[datetime.fromtimestamp(x/1000) for x in D.open_time]
    D['symbol']=symbol
    D=D[['symbol','open_date_time','open', 'high', 'low', 'close', 'volume', 'num_trades', 'taker_base_vol', 'taker_quote_vol']]

    return D

In [3]:
def highlight_cells(val,threshold=0):
    color = 'green' if val > threshold else ''
    return 'background-color: {}'.format(color)

In [4]:

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [5]:
def trend_tau(set_):
    [x1, x2] = set_
    tau, p_value = stats.kendalltau(x1,x2)
    return([tau,p_value])


In [6]:
def split_sequences(sequences, n_steps_in):

    X = list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        # check if we are beyond the dataset
        # gather input and output parts of the pattern
        seq_x = sequences[i:end_ix, :]
        X.append(seq_x)

    return np.array(X)

In [7]:

def critical_95t2(n_):
    #Studnt, n=999, p<0.05, 2-tail
    #equivalent to Excel TINV(0.05,999)

    p_v = .1
    #n_=len(future_commodities_n_w_index_pvt_w_Fred.dropna())

    t1=stats.t.ppf(1-p_v, n_-2)
    t2=stats.t.ppf(1-p_v/2, n_-2)
    #print(t1)
    #print(t2)
    crit_r1 = (t1/np.sqrt(t1*t1+n_-2))
    crit_r2 = (t2/np.sqrt(t2*t2+n_-2))
    #Studnt, n=999, p<0.05%, Single tail
    #equivalent to Excel TINV(2*0.05,999)
    #print(stats.t.ppf(1-0.05, 21-2))

    #same as t1
    """
    p_ = 0.95
    n=21
    # retrieve value <= probability
    value = stats.t.ppf(p_, 21-2)
    print(value)

    print(stats.t.ppf(1-0.025, 999))

    p_1 = stats.t.ppf(stats.t.cdf(t1, 21-2),21-2)
    print(p_1)

    p_2 = stats.t.ppf(stats.t.cdf(t2, 21-2),21-2)
    print(p_2)


    df_n = n_
    alpha = .05
    q = 1-alpha
    stats.t.ppf(q, df_n)
    """

    #print("r thresholds for t1 and t2:",crit_r1,crit_r2)
    return([crit_r1,crit_r2])

In [8]:
end_ = datetime.today()
#start = end - relativedelta(years=2) - relativedelta(days=3)
start_ = end_ - relativedelta(months=64)
start_X2_ = end_ - relativedelta(months=64*2)

nyse = mcal.get_calendar('NYSE')
nyse_dates = nyse.schedule(start_date=start_, end_date=end_)

nyse_X2 = mcal.get_calendar('NYSE')
nyse_dates_X2 = nyse.schedule(start_date=start_X2_, end_date=end_)

start = nyse_dates.index[0]
start_X2 = nyse_dates_X2.index[0]

end = nyse_dates.index[-1]
print(start.strftime('%Y-%m-%d'))
print(end.strftime('%Y-%m-%d'))

#used for trend
n_trend = 50

delta = end - start

2017-04-26
2022-08-26


In [9]:
"""
db_filename = '.\\data\\interim\\todo.db'

db_is_new = not os.path.exists(db_filename)

conn = sqlite3.connect(db_filename)

if db_is_new:
    print('Need to create schema')
else:
    print('Database exists, assume schema does, too.')

conn.close()
"""

"\ndb_filename = '.\\data\\interim\\todo.db'\n\ndb_is_new = not os.path.exists(db_filename)\n\nconn = sqlite3.connect(db_filename)\n\nif db_is_new:\n    print('Need to create schema')\nelse:\n    print('Database exists, assume schema does, too.')\n\nconn.close()\n"

In [10]:
dict_sectors = {'Basic Materials':'VAW', 'Communication Services':'VOX', 'Consumer Cyclical':'VCR',
       'Consumer Defensive':'VDC', 'Energy':'VDE', 'Financial Services':'VFH', 'Healthcare':'VHT',
       'Industrials':'VIS', 'Real Estate':'VNQ', 'Technology':'VGT', 'Utilities':'VPU', 'error':'error'}

dict_indexes = {'SPTM':'SP1500', 'SPY':'SP500', 'SLY':'SP600', 'MDYG':'SP400','error':'error'}

invert_dict_indexes = {v: k for k, v in dict_indexes.items()}

dict_sectors_reverse = {'VAW':'Basic Materials',
 'VOX':'Communication Services',
 'VCR':'Consumer Cyclical',
 'VDC':'Consumer Defensive',
 'VDE':'Energy',
 'VFH':'Financial Services',
 'VHT':'Healthcare',
 'VIS':'Industrials',
 'VNQ':'Real Estate',
 'VGT':'Technology',
 'VPU':'Utilities',
 'error': 'error'}

In [11]:
index_names = ['SP1500', 'SP500', 'SP600', 'SP400']
indexes = ['SPTM','SPY','SLY','MDYG']
sector_names = ['Communication Services ETF', 'Consumer Discretionary ETF', 'Consumer Staples ETF', 'Energy ETF', 'Financials ETF', 'Health Care ETF', 'Industrials ETF', 'Information Technology ETF', 'Materials ETF', 'Real Estate ETF', 'Utilities ETF']
sectors = ['VOX','VCR','VDC','VDE','VFH','VHT','VIS','VGT','VAW','VNQ','VPU']

etf_bonds = ['LQD', 'AGG', 'NEAR', 'IUSB', 'ISTB', 'IMTB', 'ILTB', 'GBF']
etf_muni_bonds = ['MUB', 'SUB', 'MEAR']

etf_treasuries = ['AGZ', 'GOVT', 'BIL', 'SHV', 'SHY', 'IEI', 'IEF', 'TLT']

#M1_MONEY_MULTIPLIER = M1NS/BOGMBASE

FRED_Indicators = [
"ASPUS",\
"AWHAETP",\
"B432RG3A086NBEA",\
"BAA10Y",\
"BACDINA066MNFRBNY",\
"BACTSAMFRBDAL",\
"BAMLC0A0CM",\
"BAMLCC0A1AAATRIV",\
"BAMLCC0A4BBBTRIV",\
"BAMLH0A3HYC",\
"BAMLHYH0A0HYM2TRIV",\
"BOGMBASE",\
"BOGZ1FA105015103Q",\
"BOGZ1FA145050005Q",\
"BOGZ1FA205050005Q",\
"BOGZ1FA315050005Q",\
"BOGZ1FA385050005Q",\
"BOGZ1FL105015105Q",\
"BUSAPPWNSACA",\
"BUSLOANS",\
"CASTHPI",\
"CES0500000003",\
"CES4348400001",\
"CFNAI",\
"CFNAIDIFF",\
"CFSBCACTIVITYMFG",\
"CILACBQ158SBOG",\
"CIVPART",\
"COMREPUSQ159N",\
"CONSUMER",\
"CP",\
"CPALTT01USQ657N",\
"CPIAUCSL",\
"CSCICP03USM665S",\
"CSUSHPINSA",\
"CUSR0000SEHA",\
"CUUR0000SEHA",\
"DALLCIACBEP",\
"DCOILBRENTEU",\
"DCOILWTICO",\
"DEXCHUS",\
"DEXUSUK",\
"DFF",\
"DFII10",\
"DGORDER",\
"DGS1",\
"DGS10",\
"DGS1MO",\
"DGS2",\
"DGS30",\
"DGS3MO",\
"DGS5",\
"DRBLACBS",\
"DRCCLACBS",\
"DRSFRMACBS",\
"DGS3MO",\
"DTWEXBGS",\
"EMRATIO",\
"ERENTUSQ176N",\
"ETOTALUSQ176N",\
"FEDFUNDS",\
"FGCCSAQ027S",\
"FPCPITOTLZGUSA",\
"GASREGW",\
"GDPC1",\
"GFDEBTN",\
"GFDEGDQ188S",\
"GOLDAMGBD228NLBM",\
"GVZCLS",\
"HDTGPDUSQ163N",\
"HOSMEDUSM052N",\
"IC4WSA",\
"ICSA",\
"IIPUSNETIQ",\
"INDPRO",\
"INTDSRUSM193N",\
"IRLTLT01USM156N",\
"KCFSI",\
"LES1252881600Q",\
"LEU0252918500Q",\
"LFWA64TTUSM647S",\
"LRUN64TTUSQ156S",\
"LXXRCSA",\
"M1",\
"M1V",\
"M1NS",\
"M2V",\
"MABMM301USM189S",\
"MANMM101USA189S",\
"MEFAINUSA672N",\
"MEHOINUSA672N",\
"MEPAINUSA672N",\
"MICH",\
"MOGMBASE",\
"MORTGAGE30US",\
"MPRIME",\
"MSPUS",\
"MVMTD027MNFRBDAL",\
"MZMSL",\
"NEWORDER",\
"NFCI",\
"NROU",\
"NYSTHPI",\
"PAYEMS",\
"PCE",\
"PERMIT",\
"POP",\
"POPTOTUSA647NWDB",\
"PPIACO",\
"PRFI",\
"PSAVERT",\
"Q10096USQ144NNBR",\
"RECPROUSM156N",\
"RSAHORUSQ156S",\
"SPCS20RSA",\
"STLFSI2",\
"T10Y2Y",\
"T10Y3M",\
"T10YIE",\
"T5YIE",\
"T5YIFR",\
"TB3MS",\
"TCU",\
"TDSP",\
"TEDRATE",\
"TOTALSA",\
"TREAST",\
"TTLHH",\
"TWEXB",\
"UMCSENT",\
"U6RATE",\
"UNRATE",\
"USALOLITONOSTSAM",\
"USPHCI",\
"USREC",\
"USROA",\
"USROE",\
"USSLIND",\
"USSTHPI",\
"VIXCLS",\
"VXVCLS",\
"WDFUELLA",\
"WGS3MO",\
"WILLLRGCAPVAL",\
"WILLMICROCAPPR",\
"WPU0911",\
"WPUSI019011",\
"WPU101",\
"WTB3MS"    
]

#volume
dict_treasury = {
'TreasuryYield13':'^IRX',\
'T bill Euro F':  'GE=F',\
'Two Year T F': 'ZT=F',\
'Five-Year US Treasury Note Futu':'ZF=F',\
'Ten Year T F': 'ZN=F',\
'U.S. Treasury Bond Futures':'ZB=F',\
}

#TVV
currencies = ['EURUSD=X','GBPUSD=X','AUDUSD=X','BRLUSD=X','JPY=X','GE=F','ZAR=X','CNYUSD=X']

#'World:USD':'UUP',\
dict_currencies_futures = {
    'World:USD':'DX=F',\
    'Euro:USD':'EURUSD=X',\
    'Pound:USD':'GBPUSD=X',\
    'Australia:USD':'AUDUSD=X',\
    'Real:USD':'BRLUSD=X',\
    'Yen:USD':'JPY=X',\
    'EuroDollar':'USD',\
    'Rand:USD':'ZAR=X',\
    'Yuan:USD':'CNYUSD=X',\
    'Peso:USD':'MXNUSD=X'
}

#doesn't work ,'B0=F'

indexes_futures = ['^RUT','^IRX','^GSPC']

dict_indexes_w_futures = {
    'Russel2000':'^RUT',\
    'Mini Dow Jones Indus.':'YM=F',\
    'Nasdaq 100':'NQ=F',\
    'SP500': 'ES=F',\
    'SP500':'^GSPC'
}

#'SIL=F','MGC=F','RTY=F'

other_commodities_list = ['YM=F','NQ=F','ZB=F','ZF=F','BZ=F','ZO=F','KE=F','ZR=F','ZM=F','ZL=F','GF=F','CT=F','LBS=F','OJ=F']

dict_materials_futures = {
    'Cotton':'CT=F',\
    'Random Length Lumber Futures':'LBS=F',\
}

#price
#currencies = ['EURUSD=X','GBPUSD=X','AUDUSD=X','BRLUSD=X','JPY=X','GE=F']

dict_foods = {
'Corn': 'ZC=F',\
'Soybeans': 'ZS=F',\
'Chicago Wheat': 'ZW=F',\
'Live Cattle': 'LE=F',\
'Lean Hogs': 'HE=F',\
'Cocoa': 'CC=F',\
'Sugar': 'SB=F',\
'Coffee': 'KC=F',\
'Oat Futures':'ZO=F',\
'KC HRW Wheat Futures':'KE=F',\
'Rough Rice Futures':'ZR=F',\
'Soybean Meal Futures':'ZM=F',\
'Soybean Oil Futures':'ZL=F',\
'Feeder Cattle Futures':'GF=F',\
'Orange Juice':'OJ=F',
}

#price
dict_metals = {'Gold': 'GC=F',\
'Platinum':  'PL=F',\
'Palladium': 'PA=F',\
'Silver': 'SI=F',\
'Copper':  'HG=F',\
'Aluminium': 'ALI=F'
}

#price
dict_energy = {
'Crude Oil':  'CL=F',\
'Brent Crude Oil Last Day Financ':'BZ=F',\
'Heating Oil': 'HO=F',\
'Natural Gas': 'NG=F',\
'RB Gasoline': 'RB=F'
}

etf_bonds = [*etf_bonds,*etf_muni_bonds,*etf_treasuries]

In [12]:
SP500ticker_list = pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
SP600ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies')[1]
SP400ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies')[0]


In [13]:
sp500 = SP500ticker_list['Symbol']
sp500 = [w.replace('.', '-') for w in sp500]
sp600 = SP600ticker_list['Ticker symbol']
sp600 = [w.replace('.', '-') for w in sp600]
sp400 = SP400ticker_list['Ticker symbol']
sp400 = [w.replace('.', '-') for w in sp400]


In [14]:
len(sp500)+len(sp600)+len(sp400)

1505

In [15]:
print(len(sp500))
SP1500 = list()
SP1500.extend(sp500)
sp600 = list(set(sp600).difference(SP1500))
print(len(sp600))
SP1500.extend(sp600)
sp400 = list(set(sp400).difference(SP1500))
print(len(sp400))
SP1500.extend(set(sp400).difference(SP1500))

503
601
401


In [16]:
#>.5The time series is mean reverting
#=.5 time series is a Geometric Brownian Motion
#<.5 time series is trending

def hurst(ts):
    """
    Returns the Hurst Exponent of the time series vector ts

    Parameters
    ----------
    ts : `numpy.array`
        Time series upon which the Hurst Exponent will be calculated

    Returns
    -------
    'float'
        The Hurst Exponent from the poly fit output
    """
    # Create the range of lag values
    lags = range(2, 100)

    # Calculate the array of the variances of the lagged differences
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]

    # Use a linear fit to estimate the Hurst Exponent
    poly = polyfit(log(lags), log(tau), 1)

    # Return the Hurst exponent from the polyfit output
    return poly[0]*2.0

In [17]:
sp1500_index_df = pd.DataFrame()

lists = ["sp500","sp600","sp400"]

for l in lists:
    if(l=='sp500'):
        df = pd.DataFrame(sp500,columns=['Symbol'])
        print(len(df))
        #range_ = 
        #print(range_)
        df['Market Index'] = pd.Series(["SP500" for x in range(0,len(df.index),1)])
    elif(l=='sp600'):
        df = pd.DataFrame(np.array(sp600),columns=['Symbol'])        
        print(len(df))        
        df['Market Index'] = pd.Series(["SP600" for x in range(len(df.index))])
        df.index = range(len(sp500),(len(sp500)+len(df.index)),1)        
    else:
        df = pd.DataFrame(np.array(sp400),columns=['Symbol'])
        print(len(df))
        df['Market Index'] = pd.Series(["SP400" for x in range(len(df.index))])
        df.index = range((len(sp500)+len(sp600)),(len(sp500)+len(sp600)+len(df.index)),1)
    sp1500_index_df = pd.concat([sp1500_index_df,df],axis=0)

503
601
401


In [18]:
len(sp1500_index_df)-len(SP1500)

0

In [19]:
def Fred_Data(npa):
    name = npa[0]
    start_ = npa[1]
    end_ = npa[2]
    nyse_dates_ = npa[3]
    
    temp = web.DataReader(str(name), 'fred', start_, end_)
    #temp = web.DataReader(npa[1][0],'fred',start,end)
    temp = temp.asfreq('D').interpolate(method='time',limit_direction='forward',limit_area='inside').reindex(nyse_dates_.index)
    temp.columns = ['Value']
    #temp.index = pd.to_datetime(temp.index)
    
    #temp['Symbol'] = name
    
    #print(temp)
    #temp_new = temp.asfreq('D').interpolate(method='time')    
    temp_dates = pd.date_range(temp.index[0].strftime('%Y-%m-%d'), temp.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))
    nyse_inbetween_temp_dates = nyse_dates_.index[(nyse_dates_.index >= temp_dates[0]) & (nyse_dates_.index <= temp_dates[-1])]
    temp_new = temp.reindex(nyse_inbetween_temp_dates)
    temp_new['Symbol'] = name
    
    #temp = temp.resample(frequency).mean().dropna()
    return(temp_new)
    #return(temp)

In [20]:

def dl(npa):
    name = npa[0]
    start_ = npa[1]
    end_ = npa[2]
    nyse_dates_ = npa[3]
    subset = yf.download(name, start=start_, end=end_, auto_adjust=True).iloc[:, :6].dropna(axis=0, how='any')
    
    temp_new = subset.asfreq('D').interpolate(method='time',limit_area='inside').reindex(nyse_dates_.index)
    temp_dates = pd.date_range(subset.index[0].strftime('%Y-%m-%d'), subset.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))
    nyse_inbetween_temp_dates = nyse_dates_.index[(nyse_dates_.index >= temp_dates[0]) & (nyse_dates_.index <= temp_dates[-1])]
    temp_new = temp_new.reindex(nyse_inbetween_temp_dates)
    subset_ = temp_new
    subset_['Symbol'] = name

    return (subset_)
    
def dl2(assets):

    yahoo_financials = YahooFinancials(assets)

    data = yahoo_financials.get_historical_price_data(start_date=start_date.strftime('%Y-%m-%d'), end_date=end_date.strftime('%Y-%m-%d'), time_interval='daily')
    return(data)

In [21]:
def getStock(npa):
    symbol = npa[0]
    start_=npa[1]
    end_=npa[2]
    
    data_ = yf.download(symbol, start=start_,end=end_)
  
    return([symbol,data_])

In [22]:
def getTickerData(npa):
    symbol = npa[0]
    
    stock_data = yf.Ticker(symbol) 
    return([symbol,stock_data])

In [23]:
def getSector(npa):
    #position = npa[0]
    symbol = npa[0]
    #start_=npa[2]
    #end_=npa[3]
    
    tickerdata = yf.Ticker(symbol)
    
    if 'sector' in tickerdata.info:
        sector = tickerdata.info['sector']
    else:
        sector = 'error'

    if 'industry' in tickerdata.info:
        industry = tickerdata.info['industry']
    else:
        industry = 'error'

    return([symbol,sector,industry])

In [24]:
def derive_price_supply_trends(npa_):
    name = npa_[0]
    data = npa_[1]
    nyse_dates_ = npa_[2]
        
    temp_new = data.asfreq('D').interpolate(method='time',limit_area='inside').reindex(nyse_dates_.index)
    temp_dates = pd.date_range(data.index[0].strftime('%Y-%m-%d'), data.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))
    nyse_inbetween_temp_dates = nyse_dates_.index[(nyse_dates_.index >= temp_dates[0]) & (nyse_dates_.index <= temp_dates[-1])]
    temp_new = temp_new.reindex(nyse_inbetween_temp_dates)
    
    data_ = temp_new
    data_['risk_50td_trend'] = (data.shift(50)-data)/ ((data.rolling(50).max()-data.rolling(50).min()) /4) 
    data_['Symbol'] = name
    return(data_)
    

In [25]:
def derive_tvv(npa_):
    name = npa_[0]
    data = npa_[1]
    #print(data)
    nyse_dates_ = npa_[2]
        
    temp_new = data.asfreq('D').interpolate(method='time',limit_area='inside').reindex(nyse_dates_.index)
    temp_dates = pd.date_range(data.index[0].strftime('%Y-%m-%d'), data.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))
    nyse_inbetween_temp_dates = nyse_dates_.index[(nyse_dates_.index >= temp_dates[0]) & (nyse_dates_.index <= temp_dates[-1])]
    temp_new = temp_new.reindex(nyse_inbetween_temp_dates)
    
    data_ = temp_new
    data_['Symbol'] = name

    """
    if(np.sum(duplicates==names_prices[n])>0):
        #potential duplicate
        if((np.sum((np.unique(prices_df['Symbol']))==n))>0):
            pass
        else:
            prices_df = pd.concat([prices_df,data_],axis=0)            
    else:
    """
    
    data_['VWAP'] = (data_['Adj Close']*data_['Volume'])
    
    data_['buy_volume_pct'] = (data_['Adj Close']-data_['Low'])/(data_['High']-data_['Low'])
    data_['sell_volume_pct'] = (data_['Adj Close']-data_['High'])/(data_['High']-data_['Low'])



    data_['daily_buy_vol'] = data_['buy_volume_pct']*data_['Volume']
        
        
    
    data_['daily_sell_vol'] = data_['sell_volume_pct']*data_['Volume']
    
    data_['net_daily_vol'] = data_['daily_buy_vol']+data_['daily_sell_vol']
    
    data_['20td_trend_vol'] = data_['net_daily_vol'].rolling(20).sum()
    
    
    data_['20td_tot_daily_vol'] = data_['Volume'].rolling(20).sum()
    
    data_['20td_trend_vol_factor'] = data_['20td_trend_vol']/data_['20td_tot_daily_vol']
    
    data_['20td_tvf_rng_div_2'] = (data_['20td_trend_vol_factor'].rolling(20).max()-data_['20td_trend_vol_factor'].rolling(20).min())/2
    
    data_['20td_tvf_vel'] = data_['20td_trend_vol_factor']/data_['20td_tvf_rng_div_2']    
    
    
    data_['50td_trend_vol'] = data_['net_daily_vol'].rolling(20).sum()
    
    data_['50td_tot_daily_vol'] = data_['Volume'].rolling(20).sum()
    
    data_['50td_trend_vol_factor'] = data_['50td_trend_vol']/data_['50td_tot_daily_vol']
    
    data_['50td_tvf_rng_div_4'] = (data_['50td_trend_vol_factor'].rolling(20).max()-data_['50td_trend_vol_factor'].rolling(20).min())/4
    
    data_['50td_tvf_vel'] = data_['50td_trend_vol_factor']/data_['50td_tvf_rng_div_4']    
    
    
    data_['trailing_1yr_max'] = (pd.DataFrame(data_)['High']).rolling(252).max()
    data_['MIN_Lookback_One_Year'] = data_['trailing_1yr_max'].rolling(252).min()
    #data_['MIN_Lookback_6_Months'] = data_['trailing_1yr_max'].rolling(126).min()
    data_['MIN_Lookback_2_Months'] = data_['trailing_1yr_max'].rolling(42).min()
    data_['MIN_Lookback_1_Months'] = data_['trailing_1yr_max'].rolling(21).min()

    data_['trailing_1yr_min'] = (pd.DataFrame(data_)['Low']).rolling(252).min()
    
    data_['supply_trend_1yr'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(252).min()
    
    #data_['supply_trend_180d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(126).min()
    
    #data_['supply_trend_200td'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(63).min()
    
    data_['supply_trend_60d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(42).min()
    
    data_['supply_trend_30d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(21).min()

    data_['trailing_60d_max'] = (pd.DataFrame(data_)['High']).rolling(42).max()
    data_['trailing_60d_min'] = (pd.DataFrame(data_)['Low']).rolling(42).min()
    
    data_['trailing_30d_max'] = (pd.DataFrame(data_)['High']).rolling(21).max()
    data_['trailing_30d_min'] = (pd.DataFrame(data_)['Low']).rolling(21).min()
    
    data_['200tdSMA'] = (pd.DataFrame(data_)['Adj Close']).rolling(200).mean()
    
    data_['200tdSDev'] = (pd.DataFrame(data_)['Adj Close']).rolling(200).std()
    
    data_['TP'] = (data_['Adj Close'] + data_['Low'] + data_['High'])/3
    data_['50tdstdTP'] = data_['TP'].rolling(50).std(ddof=0)
    data_['50tdMA-TP'] = data_['TP'].rolling(50).mean()
    data_['50tdBOLU'] = data_['50tdMA-TP'] + 2*data_['50tdstdTP']
    data_['50tdBOLD'] = data_['50tdMA-TP'] - 2*data_['50tdstdTP']    

    data_['trailing_risk_40d_max'] = (pd.DataFrame(data_)['High']).rolling(40).max()
    data_['trailing_risk_40d_min'] = (pd.DataFrame(data_)['Low']).rolling(40).min()
    
    data_['risk_per_share'] = data_['trailing_risk_40d_max']-data_['trailing_risk_40d_min']
    

    data_['30d_vol'] = (pd.DataFrame(data_)['Volume']).rolling(21).sum()
        
    data_['30d_vol_2yr'] = data_['30d_vol'].rolling(512).mean()
    data_['volume_factor'] = data_['30d_vol']/data_['30d_vol_2yr']
    
    #data_['1YMF'] = (data_['Adj Close']-data_['MIN_Lookback_One_Year']) + (data_['Adj Close']-data_['MIN_Lookback_6_Months']) + (data_['Adj Close']-data_['MIN_Lookback_2_Months']) + (data_['Adj Close']-data_['MIN_Lookback_1_Months'])
    data_['1YMF'] = (data_['Adj Close']-data_['MIN_Lookback_One_Year']) + (data_['Adj Close']-data_['MIN_Lookback_2_Months']) + (data_['Adj Close']-data_['MIN_Lookback_1_Months'])
    
    data_['risk_trend_factor'] = data_['1YMF']/data_['risk_per_share']
    
    return(data_)

In [26]:
def calculateMetrics(data_):

    df_line = data_[['Symbol','buy_volume_pct','sell_volume_pct','daily_buy_vol','daily_sell_vol','net_daily_vol','20td_trend_vol','20td_tot_daily_vol','20td_trend_vol_factor','20td_tvf_rng_div_2','20td_tvf_vel','50td_trend_vol','50td_tot_daily_vol','50td_trend_vol_factor','50td_tvf_rng_div_4','50td_tvf_vel','MIN_Lookback_One_Year', 'MIN_Lookback_2_Months', 'MIN_Lookback_1_Months','risk_trend_factor','risk_per_share','volume_factor','Adj Close','50tdBOLD','50tdMA-TP','50tdBOLU','200tdSMA','200tdSDev']].iloc[[-1]].reset_index().set_index('Symbol')
    name = df_line.index[0]
    adf = ts.adfuller(data_['Adj Close'].tail(n_trend).values)[1]

    hurst_ = hurst(data_['Adj Close'].tail(n_trend).values)    

    data = data_['Adj Close'].tail(n_trend)
    dates = [pd.to_datetime(d).value for d in data.index]

    dates = np.array(dates)-np.array(dates[0])/10**11

    int_date_data = pd.concat([pd.DataFrame(dates),pd.DataFrame(data.values)],axis=1)

    trend_t = [*[trend_tau([pd.DataFrame(int_date_data.iloc[:,0]).values,data.values])]]

    tau = trend_t[0][0]
    tau_p = trend_t[0][1]

    try:
        adf = ts.adfuller(data_['Adj Close'].tail(n_trend).values)[1]
    except:
        adf = 'error'

    try:
        hurst_ = hurst(data_['Adj Close'].tail(n_trend).values)
    except:
        hurst_ = 'error'

    temp = pd.concat([df_line,pd.DataFrame([adf],index=[name],columns=['adf_50td']),pd.DataFrame([tau],index=[name],columns=['tau_50td']),pd.DataFrame([tau_p],index=[name],columns=['tau_50td_p']),pd.DataFrame([hurst_],index=[name],columns=['hurst_50td'])],axis=1)
    temp.columns = ['Date',*temp.columns[1:]]
    temp

    return(temp)
    

In [27]:
def extractTickerValues(npa):
    
    n_ = npa[0]
    r_ = npa[1]
    #print(r_)
    print(n_)

    subset = pd.DataFrame([r_.info])

    others = ['quarterly_balance_sheet','quarterly_cashflow','quarterly_dividends','quarterly_earnings','quarterly_financials','mutualfund_holders','options','sustainability']

    o_items = []

    #print(r_)
    for o in others:
        if(np.sum(np.array(dir(r_))==[o])>0):
            o_item = getattr(r_, o)
            o_items.append([o,o_item])            

    
    values = pd.DataFrame()

    trackers = ['shortName','beta','currentRatio','debtToEquity','dividendRate','dividendYield','earningsGrowth','earningsQuarterlyGrowth','ebitda','ebitdaMargins','exDividendDate','forwardEPS','forwardPE','freeCashflow','grossMargins','grossProfits','heldPercentInstitutions','industry','lastDividendDate','lastDividendValue','lastSplitDate','lastSplitFactor','marketCap','operatingCashflow','operatingMargins','payoutRatio','pegRatio','priceToBook','priceToSalesTrailing12Months','profitMargins','quickRatio','returnOnAssets','returnOnEquity','revenueGrowth','revenuePerShare','sector','sharesOutstanding','shortRatio','totalAssets','totalCash','totalCashPerShare','totalDebt','totalRevenue','trailingEPS','trailingAnnualDividendRate','trailingAnnualDividendYield','trailingPE']

    for t in trackers:
        if(np.sum(np.where(subset.columns==t))==0):
            values = pd.concat([values,pd.DataFrame('error',index=[n_],columns=[t])],axis=1)
        else:
            values = pd.concat([values,pd.DataFrame(subset[t].values[0],index=[n_],columns=[t])],axis=1)

    return([n_,values,o_items])

In [28]:
tvv_extracted_list = np.unique([*[dict_indexes_w_futures[d] for d in dict_indexes_w_futures],*[dict_materials_futures[d] for d in dict_materials_futures],*[dict_foods[d] for d in dict_foods],*[dict_currencies_futures[d] for d in dict_currencies_futures],*[dict_treasury[d] for d in dict_treasury],*[dict_metals[d] for d in dict_metals],*[dict_energy[d] for d in dict_energy]])
npa = []
for f in tvv_extracted_list:
    npa.append([f,start_X2,end])

print("npa length:getStock(commodites):",len(npa))

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()
future = client.map(getStock, npa)
results_commodities_futures = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_commodities_futures.append("error")
    else:
        results_commodities_futures.append(f.result()) 
client.close()

"""
data_['trailing_risk_40d_max'] = (pd.DataFrame(data_)['High']).rolling(40).max()
data_['trailing_risk_40d_min'] = (pd.DataFrame(data_)['Low']).rolling(40).min()
"""

npa length:getStock(commodites): 48


C:\Users\User\AppData\Local\Programs\3.9-JupyterLab\lib\site-packages\distributed\client.py:1131: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | 1.10.6 | None      | None    |
| lz4     | 4.0.2  | 3.1.10    | 3.1.10  |
| tornado | 6.2    | 6.1       | 6.1     |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


"\ndata_['trailing_risk_40d_max'] = (pd.DataFrame(data_)['High']).rolling(40).max()\ndata_['trailing_risk_40d_min'] = (pd.DataFrame(data_)['Low']).rolling(40).min()\n"

In [29]:
#price_list = [*[dict_currencies_futures[d] for d in dict_currencies_futures],*[dict_treasury[d] for d in dict_treasury],*[dict_metals[d] for d in dict_metals],*[dict_energy[d] for d in dict_energy]]


In [30]:
names_comm_results = [r[0] for r in results_commodities_futures]
sum_comm_tvv_volume = [np.sum(r[1]['Volume']) for r in results_commodities_futures]

In [31]:
no_volume_comm = np.array(names_comm_results)[np.array(sum_comm_tvv_volume)==0]


In [32]:
#other_commodities_list_df[other_commodities_list_df['Symbol']=='ALI=F']

In [33]:
list_comm_output = [pd.DataFrame(r[1]) for r in results_commodities_futures]
list_comm_prices = [r[0] for r in results_commodities_futures]
dict_comm_prices = dict(zip(*[list_comm_prices,list_comm_output])) 


In [34]:
"""
full_commodities_price_df_pvt = pd.DataFrame()

for r_df in results_commodities_futures:

        temp_df = r_df[1].asfreq('D').interpolate(method='time',limit_direction='forward',limit_area='inside').reindex(nyse_dates_X2.index)
        temp_dates = pd.date_range(temp_df.index[0].strftime('%Y-%m-%d'), temp_df.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))
        nyse_inbetween_temp_dates = nyse_dates_X2.index[(nyse_dates_X2.index >= temp_dates[0]) & (nyse_dates_X2.index <= temp_dates[-1])]
        temp_new = temp_df.reindex(nyse_inbetween_temp_dates)
        #temp_new['Symbol'] = r_df[0]
        price = temp_new[['Adj Close']]
        price.columns = [r_df[0]]
        print(price)
        full_commodities_price_df_pvt = pd.concat([price.copy(),full_commodities_price_df_pvt],axis=1)
        
#full_commodities_df_pvt = pd.pivot_table(full_commodities_df_pvt, values=['Adj Close'], index=other_commodities_list_df.index,columns=['Symbol'])
#full_commodities_df_pvt.columns = full_commodities_df_pvt.columns.droplevel(0)        
"""

"\nfull_commodities_price_df_pvt = pd.DataFrame()\n\nfor r_df in results_commodities_futures:\n\n        temp_df = r_df[1].asfreq('D').interpolate(method='time',limit_direction='forward',limit_area='inside').reindex(nyse_dates_X2.index)\n        temp_dates = pd.date_range(temp_df.index[0].strftime('%Y-%m-%d'), temp_df.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))\n        nyse_inbetween_temp_dates = nyse_dates_X2.index[(nyse_dates_X2.index >= temp_dates[0]) & (nyse_dates_X2.index <= temp_dates[-1])]\n        temp_new = temp_df.reindex(nyse_inbetween_temp_dates)\n        #temp_new['Symbol'] = r_df[0]\n        price = temp_new[['Adj Close']]\n        price.columns = [r_df[0]]\n        print(price)\n        full_commodities_price_df_pvt = pd.concat([price.copy(),full_commodities_price_df_pvt],axis=1)\n        \n#full_commodities_df_pvt = pd.pivot_table(full_commodities_df_pvt, values=['Adj Close'], index=other_commodities_list_df.index,columns=['Symbol'])

In [35]:
"""
for d in [d for d in dict_comm_prices]:
    #print(d)
    plt.plot(derive_tvv([d,dict_comm_prices[d],nyse_dates_X2])[['50td_tvf_vel']])
    plt.show()
    #d = dict_comm_prices[d][0]
    #print(d)
"""

"\nfor d in [d for d in dict_comm_prices]:\n    #print(d)\n    plt.plot(derive_tvv([d,dict_comm_prices[d],nyse_dates_X2])[['50td_tvf_vel']])\n    plt.show()\n    #d = dict_comm_prices[d][0]\n    #print(d)\n"

In [36]:
"""
#d= [d for d in dict_comm_prices][0]
comm_tvv_df_pvt = pd.DataFrame()

for d in [d for d in dict_comm_prices]:
    #print(d)    
    test = derive_tvv([d,dict_comm_prices[d],nyse_dates_X2])[['50td_tvf_vel']]
        
    #if(np.sum(dict_comm_p
    if(abs(np.sum(test))[0]>0):
        if((len(test.dropna()))<int(0.5*test.shape[0])):
            temp = derive_price_supply_trends([d,dict_comm_prices[d][['Adj Close']],nyse_dates_X2])[['risk_50td_trend']]
        else:
            temp = derive_tvv([d,dict_comm_prices[d],nyse_dates_X2])[['50td_tvf_vel']]            
    else:
        temp = derive_price_supply_trends([d,dict_comm_prices[d][['Adj Close']],nyse_dates_X2])[['risk_50td_trend']]
    
    temp.columns = [d]
    
    temp = temp.asfreq('D').interpolate().asfreq('W-'+end.strftime('%a'))
        
    comm_tvv_df_pvt = pd.concat([comm_tvv_df_pvt,temp],axis=1)
"""

"\n#d= [d for d in dict_comm_prices][0]\ncomm_tvv_df_pvt = pd.DataFrame()\n\nfor d in [d for d in dict_comm_prices]:\n    #print(d)    \n    test = derive_tvv([d,dict_comm_prices[d],nyse_dates_X2])[['50td_tvf_vel']]\n        \n    #if(np.sum(dict_comm_p\n    if(abs(np.sum(test))[0]>0):\n        if((len(test.dropna()))<int(0.5*test.shape[0])):\n            temp = derive_price_supply_trends([d,dict_comm_prices[d][['Adj Close']],nyse_dates_X2])[['risk_50td_trend']]\n        else:\n            temp = derive_tvv([d,dict_comm_prices[d],nyse_dates_X2])[['50td_tvf_vel']]            \n    else:\n        temp = derive_price_supply_trends([d,dict_comm_prices[d][['Adj Close']],nyse_dates_X2])[['risk_50td_trend']]\n    \n    temp.columns = [d]\n    \n    temp = temp.asfreq('D').interpolate().asfreq('W-'+end.strftime('%a'))\n        \n    comm_tvv_df_pvt = pd.concat([comm_tvv_df_pvt,temp],axis=1)\n"

In [37]:

#price_comm_df_10_yr_pvt_corr.to_excel(writer, sheet_name='price_comm_df_10_yr_pvt_corr')
#price_comm_df_5_yr_pvt_corr.to_excel(writer, sheet_name='price_comm_df_5_yr_pvt_corr')

#future_commodities_price_pvt = pd.pivot_table(commodity_prices, values=['Adj Close'], index=commodity_prices.index,columns=['Symbol'])
#future_commodities_price_pvt.columns = future_commodities_price_pvt.columns.droplevel(0)

In [38]:


#METALS CORRELATION MATRIX doing the same thing with (1) 50 day price change and (2) 50 day TVV.  Please run them all against each other to complete the METALS correlation matrix.  Run a 5 year and 10 year Matrix

#future_commodities_price_pvt_corr = future_commodities_price_pvt.corr().replace([np.NaN],0)

#future_commodities_50td_tvv_pvt.columns = full_commodities_df_pvt.columns.droplevel(0)

#'MVMTD027MNFRBDAL'
#check_corr_list = ['^GSPC','DGS3MO','DGS2','DGS10']
#check_corr_list = [dict_metals[d] for d in dict_metals]

"""
for c_c in check_corr_list:
    #inside = future_commodities_50td_tvv_pvt_corr.sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)
    inside = comm_tvv_df_10_yr_pvt[dict_metals[d] for d in dict_metals].corr().sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)
    plt.figure(figsize=(24,15))
    sns.heatmap(inside,cmap="RdYlGn",annot=True)
    plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'comm_tvv_df_10_yr_pvt_corr.png', dpi=300, format='png', bbox_inches='tight')
    plt.show()
    
"""


#price_comm_df_10_yr_pvt_corr[price_comm_df_10_yr_pvt_corr.columns.intersection([dict_metals[d] for d in dict_metals])].corr().to_excel(writer, sheet_name='price_comm_metals_10_yr_corr')
#price_comm_df_5_yr_pvt_corr[price_comm_df_5_yr_pvt_corr.columns.intersection([dict_metals[d] for d in dict_metals])].corr().to_excel(writer, sheet_name='price_comm_metals_5_yr_corr')

'\nfor c_c in check_corr_list:\n    #inside = future_commodities_50td_tvv_pvt_corr.sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)\n    inside = comm_tvv_df_10_yr_pvt[dict_metals[d] for d in dict_metals].corr().sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)\n    plt.figure(figsize=(24,15))\n    sns.heatmap(inside,cmap="RdYlGn",annot=True)\n    plt.savefig(\'..\\reports\\figures\\\'+end.strftime(\'%Y-%m-%d\')+\'-\'+c_c+\'comm_tvv_df_10_yr_pvt_corr.png\', dpi=300, format=\'png\', bbox_inches=\'tight\')\n    plt.show()\n    \n'

In [39]:
#price_comm_df_10_yr_pvt_corr[set[price_comm_df_10_yr_pvt_corr.columns].intersection(set(currencies))].corr().to_excel(writer, sheet_name='price_comm_curr_10_yr_corr')
#price_comm_df_5_yr_pvt_corr[set[price_comm_df_5_yr_pvt_corr.columns].intersection(set(currencies))].corr().to_excel(writer, sheet_name='price_comm_curr_5_yr_corr')

In [40]:
#price_comm_df_10_yr_pvt_corr[price_comm_df_10_yr_pvt_corr.columns.intersection([dict_foods[d] for d in dict_foods])].corr().to_excel(writer, sheet_name='price_comm_foods_10_yr_corr')
#price_comm_df_5_yr_pvt_corr[price_comm_df_5_yr_pvt_corr.columns.intersection([dict_foods[d] for d in dict_foods])].corr().to_excel(writer, sheet_name='price_comm_foods_5_yr_corr')

In [41]:
#price_comm_df_10_yr_pvt_corr[price_comm_df_10_yr_pvt_corr.columns.intersection([dict_energy[d] for d in dict_energy])].corr().to_excel(writer, sheet_name='price_comm_energy_10_yr_corr')
#price_comm_df_5_yr_pvt_corr[price_comm_df_5_yr_pvt_corr.columns.intersection([dict_energy[d] for d in dict_energy])].corr().to_excel(writer, sheet_name='price_comm_energy_5_yr_corr')

In [42]:
"""
price_comm_df_10_yr_pvt_corr.to_excel(writer, sheet_name='price_comm_df_10_yr_pvt_corr')
price_comm_df_5_yr_pvt_corr.to_excel(writer, sheet_name='comm_t')
pd.DataFrame(sp400).to_excel(writer, sheet_name='SP400')
pd.DataFrame(market_data).to_excel(writer, sheet_name='Stock_Fundamentals')
completed_fred_pvt.to_excel(writer, sheet_name='completed_fred_pvt')
completed_bonds.to_excel(writer, sheet_name='completed_bonds')
completed_bonds_pvt.to_excel(writer, sheet_name='completed_bonds_pvt')
pd.DataFrame(pd.DataFrame(sectors,sector_names)).to_excel(writer, sheet_name='vanguard_sectors')
pd.DataFrame(pd.DataFrame(indexes,index_names)).to_excel(writer, sheet_name='spdr_indexes')
screener_metrics.to_excel(writer, sheet_name='Screener')

writer.save()

wb = openpyxl.load_workbook('SP1500_screener_multiple.xlsx')
#openpyxl.Workbook()
new_range = openpyxl.workbook.defined_name.DefinedName('Prices', attr_text='SP1500_Sectors_Indexes_Prices!$A$1:$H$'+str(len(prices_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SP1500', attr_text='SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('Vanguard_Sectors', attr_text='vanguard_sectors!$B$1:$C$'+str(len(sectors)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SPDR_Indexes', attr_text='spdr_indexes!$B$1:$C$'+str(len(indexes)+1))
wb.defined_names.append(new_range)


#wb.defined_names.append(new_range)
wb.save('SP1500_screener_multiple.xlsx')
wb.close()
#named_range = NamedRange("test_nr", [(new_sheet, "A1")])
#wb.add_named_range(new_range)
wb.close()

"""
"""
workbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')
workbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))
workbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))
workbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))
workbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))

workbook.close()
"""

"\nworkbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')\nworkbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))\nworkbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))\nworkbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))\nworkbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))\n\nworkbook.close()\n"

In [43]:
#TVV

"""
tvv_corr_set = np.unique([*[dict_treasury[d] for d in dict_treasury],*[d for d in other_commodities_list],*[d for d in indexes_futures]])
"""

'\ntvv_corr_set = np.unique([*[dict_treasury[d] for d in dict_treasury],*[d for d in other_commodities_list],*[d for d in indexes_futures]])\n'

In [44]:
"""
list1_as_set = set(tvv_corr_set)

intersection = comm_tvv_df_10_yr_pvt_corr.columns.intersection(list1_as_set)
intersection_as_list = list(intersection)

comm_tvv_df_10_yr_pvt_corr.loc[intersection_as_list][intersection_as_list].to_excel(writer, sheet_name='comm_tvv_10_yr_corr')

list1_as_set = set(tvv_corr_set)
intersection = comm_tvv_df_5_yr_pvt_corr.columns.intersection(list1_as_set)
intersection_as_list = list(intersection)

comm_tvv_df_5_yr_pvt_corr.loc[intersection_as_list][intersection_as_list].to_excel(writer, sheet_name='comm_tvv_5_yr_corr')

"""

"\nlist1_as_set = set(tvv_corr_set)\n\nintersection = comm_tvv_df_10_yr_pvt_corr.columns.intersection(list1_as_set)\nintersection_as_list = list(intersection)\n\ncomm_tvv_df_10_yr_pvt_corr.loc[intersection_as_list][intersection_as_list].to_excel(writer, sheet_name='comm_tvv_10_yr_corr')\n\nlist1_as_set = set(tvv_corr_set)\nintersection = comm_tvv_df_5_yr_pvt_corr.columns.intersection(list1_as_set)\nintersection_as_list = list(intersection)\n\ncomm_tvv_df_5_yr_pvt_corr.loc[intersection_as_list][intersection_as_list].to_excel(writer, sheet_name='comm_tvv_5_yr_corr')\n\n"

In [45]:
"""
list1_as_set = set(tvv_corr_set)
intersection = comm_tvv_df_10_yr_pvt_corr.columns.intersection(list1_as_set)
intersection_as_list = list(intersection)


for d in [d for d in dict_treasury]:
    #print(d)
    d = dict_treasury[d]
    print(d)
    
    inside_10 = comm_tvv_df_10_yr_pvt_corr[intersection_as_list].sort_values(by=d,ascending=False,axis=0).sort_values(by=d,ascending=False,axis=1)
    inside_5 = comm_tvv_df_5_yr_pvt_corr[intersection_as_list].sort_values(by=d,ascending=False,axis=0).sort_values(by=d,ascending=False,axis=1)
    
    inside_10[intersection_as_list].to_excel(writer, sheet_name=d+'_comm_tvv_10_yr_corr')
    inside_5[intersection_as_list].to_excel(writer, sheet_name=d+'_comm_tvv_5_yr_corr')

    #print('10 yr')
    #plt.figure(figsize=(24,15))
    #sns.heatmap(inside_10,cmap="RdYlGn",annot=True)
    #plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+d+'price_comm_df_10_yr_pvt_corr.png', dpi=300, format='png', bbox_inches='tight')
    #plt.show()
    #print('5 yr')
    #plt.figure(figsize=(24,15))
    #sns.heatmap(inside_5,cmap="RdYlGn",annot=True)
    #plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+d+'price_comm_df_5_yr_pvt_corr.png', dpi=300, format='png', bbox_inches='tight')
    #plt.show()
    
"""

"""    
for 
#price
#price_corr_set = 
[dict_energy[d] for d in dict_energy]
[dict_metals[d] for d in dict_metals]
[currencies[d] for d in currencies]

[dict_foods[d] for d in dict_foods]
"""

'    \nfor \n#price\n#price_corr_set = \n[dict_energy[d] for d in dict_energy]\n[dict_metals[d] for d in dict_metals]\n[currencies[d] for d in currencies]\n\n[dict_foods[d] for d in dict_foods]\n'

In [46]:

tbill_3mo_df = Fred_Data(['DGS3MO',start_X2,end,nyse_dates_X2])[['Value']]
tbill_3mo_df.columns = ['DGS3MO']

tbill_2y_df = Fred_Data(['DGS2',start_X2,end,nyse_dates_X2])[['Value']]
tbill_2y_df.columns = ['DGS2']

tbill_10y_df = Fred_Data(['DGS10',start_X2,end,nyse_dates_X2])[['Value']]
tbill_10y_df.columns = ['DGS10']


In [47]:
#for c_c in check_corr_list:
    #future_commodities_50td_tvv_pvt = pd.concat([future_commodities_50td_tvv_pvt,future_commodities_n_w_index_pvt_w_Fred[[c_c]]],axis=1)


In [48]:
"""
future_commodities_50td_tvv_pvt_corr = future_commodities_50td_tvv_pvt.corr().replace([np.NaN],0)

#future_commodities_50td_tvv_pvt.columns = full_commodities_df_pvt.columns.droplevel(0)

#'MVMTD027MNFRBDAL'
#check_corr_list = ['^GSPC','DGS3MO','DGS2','DGS10']
check_corr_list = [dict_treasury[d] for d in dict_treasury]

for c_c in check_corr_list:
    #inside = future_commodities_50td_tvv_pvt_corr.sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)
    inside = comm_tvv_df_10_yr_pvt.corr().sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)
    plt.figure(figsize=(24,15))
    sns.heatmap(inside,cmap="RdYlGn",annot=True)
    plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'comm_tvv_df_10_yr_pvt_corr.png', dpi=300, format='png', bbox_inches='tight')
    plt.show()
"""

'\nfuture_commodities_50td_tvv_pvt_corr = future_commodities_50td_tvv_pvt.corr().replace([np.NaN],0)\n\n#future_commodities_50td_tvv_pvt.columns = full_commodities_df_pvt.columns.droplevel(0)\n\n#\'MVMTD027MNFRBDAL\'\n#check_corr_list = [\'^GSPC\',\'DGS3MO\',\'DGS2\',\'DGS10\']\ncheck_corr_list = [dict_treasury[d] for d in dict_treasury]\n\nfor c_c in check_corr_list:\n    #inside = future_commodities_50td_tvv_pvt_corr.sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)\n    inside = comm_tvv_df_10_yr_pvt.corr().sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)\n    plt.figure(figsize=(24,15))\n    sns.heatmap(inside,cmap="RdYlGn",annot=True)\n    plt.savefig(\'..\\reports\\figures\\\'+end.strftime(\'%Y-%m-%d\')+\'-\'+c_c+\'comm_tvv_df_10_yr_pvt_corr.png\', dpi=300, format=\'png\', bbox_inches=\'tight\')\n    plt.show()\n'

In [49]:
"""

check_corr_list = ['^GSPC','DGS3MO','DGS2','DGS10']

for c_c in check_corr_list:

    data = future_commodities_n_w_index_pvt_w_Fred[[c_c]]
    dates = [pd.to_datetime(d).value for d in data.index]

    dates = np.array(dates)-np.array(dates[0])/10**11

    int_date_data = pd.concat([pd.DataFrame(dates),pd.DataFrame(data[[c_c]].values)],axis=1)
    int_date_data.columns = ['Date',c_c]

    windows = split_sequences(int_date_data.values,n_trend)
    windows = pd.DataFrame(windows).head(-(n_trend-1)).values.ravel()        

    trend_t = [*[trend_tau([pd.DataFrame(d).iloc[:,0].values,pd.DataFrame(d).iloc[:,1].values]) for d in windows]]
    sp_w_trend = pd.concat([pd.DataFrame(data[c_c].values),pd.DataFrame(np.append(np.repeat(np.nan,len(data)-len([t[0] for t in trend_t])),[t[0] for t in trend_t])),pd.DataFrame(np.append(np.repeat(np.nan,len(data)-len([t[0] for t in trend_t])),[t[0] for t in trend_t]))],axis=1)
    sp_w_trend.columns = [c_c,'trending','p-value']
    sp_w_trend.index = data.index
    trending = sp_w_trend[['trending']]>0  
    #trending = pd.DataFrame((future_commodities_n_w_index_pvt_w_Fred[c_c]-future_commodities_n_w_index_pvt_w_Fred[c_c].shift(n_trend))>0)

    trending.columns = ['trending']
    trending_df = pd.concat([future_commodities_n_w_index_pvt_w_Fred,trending],axis=1)

    future_commodities_trending = trending_df[trending_df['trending']==True]
    future_commodities_not_trending = trending_df[trending_df['trending']!=True]

    sorted_by_c_future_commodities_corr = future_commodities_n_w_index_pvt_w_Fred.corr().sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)
    sorted_by_c_future_commodities_corr.to_csv('..\\data\\processed\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_sorted_by_c_future_commodities_corr.csv')

    future_commodities_trending_corr = future_commodities_trending.corr().sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)
    sorted_by_c_future_commodities_corr.to_csv('..\\data\\processed\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_future_commodities_trending_corr.csv')

    future_commodities_not_trending_corr = future_commodities_not_trending.corr().sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)
    future_commodities_not_trending_corr.to_csv('..\\data\\processed\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_future_commodities_not_trending_corr.csv')

    plt.figure(figsize=(24,15))
    sns.heatmap(future_commodities_n_w_index_pvt_w_Fred[sorted_by_c_future_commodities_corr.index].corr(),cmap="RdYlGn",annot=True)
    plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_sorted_by_c_future_commodities_corr.png', dpi=300, format='png', bbox_inches='tight')
    #plt.show()

    fig, ax1 = plt.subplots( figsize=(15,6) )

    ax2 = ax1.twinx()

    ax1.plot(trending_df[[c_c]])
    ax2.plot(trending_df[['trending']])
    ax2.fill_between(trending_df.index, 0, 1, where=trending_df['trending'] == True,facecolor='blue', alpha=0.5)
    plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_trending.png', dpi=300, format='png', bbox_inches='tight')
    #plt.show()

    print("correlations when trending")
    print(len(future_commodities_trending))

    sorted_future_commodities_trending = future_commodities_trending.corr()[[c_c]].sort_values(by=c_c,ascending=False)

    plt.figure(figsize=(24,15))
    sns.heatmap(future_commodities_trending[sorted_future_commodities_trending.index].corr(),cmap="RdYlGn",annot=True)
    plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_sorted_by_c_future_commodities_trending_corr.png', dpi=300, format='png', bbox_inches='tight')
    #plt.show()

    print("correlations when not trending")
    print(len(future_commodities_not_trending))

    sorted_future_commodities_not_trending = future_commodities_not_trending.corr()[[c_c]].sort_values(by=c_c,ascending=False)

    plt.figure(figsize=(24,15))
    sns.heatmap(future_commodities_not_trending[sorted_future_commodities_not_trending.index].corr(),cmap="RdYlGn",annot=True)
    plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_sorted_by_c_future_commodities_not_trending_corr.png', dpi=300, format='png', bbox_inches='tight')
    #plt.show()

    plt.figure(figsize=(4,5))
    sns.heatmap((sorted_future_commodities_not_trending-sorted_future_commodities_trending).sort_values(by=[c_c],ascending=False),cmap="RdYlGn",annot=True)
    plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_sorted_by_c_future_commodities_trending-not_trending_corr.png', dpi=300, format='png', bbox_inches='tight')
    #plt.show()

    n = len(future_commodities_not_trending)

    dist = scipy.stats.beta(n/2 - 1, n/2 - 1, loc=-1, scale=2)
    p_not_trending = pd.DataFrame(2*dist.cdf(-abs(future_commodities_not_trending[sorted_future_commodities_not_trending.index].corr())),columns=future_commodities_not_trending.columns,index=sorted_future_commodities_not_trending.index)

    n = len(future_commodities_trending)

    dist = scipy.stats.beta(n/2 - 1, n/2 - 1, loc=-1, scale=2)
    p_trending = pd.DataFrame(2*dist.cdf(-abs(future_commodities_trending[sorted_future_commodities_trending.index].corr())),columns=future_commodities_trending.columns,index=sorted_future_commodities_trending.index)
    #p_trending = pd.DataFrame([2*dist.cdf(-abs(future_commodities_trending[sorted_future_commodities_trending.index].corr()))],columns=sorted_future_commodities_trending.columns,index=sorted_future_commodities_trending.index)


    temp_all_significant = pd.DataFrame((pd.DataFrame(p_not_trending)<.05)*(pd.DataFrame(p_trending)<.05)*(future_commodities_not_trending[sorted_future_commodities_not_trending.index].corr()*future_commodities_trending[sorted_future_commodities_trending.index].corr())>0)
    temp_all_significant_inverted = pd.DataFrame((pd.DataFrame(p_not_trending)<.05)*(pd.DataFrame(p_trending)<.05)*(future_commodities_not_trending[sorted_future_commodities_not_trending.index].corr()*future_commodities_trending[sorted_future_commodities_trending.index].corr())<0)
    temp_trend_significant = pd.DataFrame(pd.DataFrame(p_trending)<.05)
    temp_not_trend_significant = pd.DataFrame(pd.DataFrame(p_not_trending)<.05)

    combined = pd.concat([temp_trend_significant.loc[[c_c]].T,temp_not_trend_significant.loc[[c_c]].T,temp_all_significant.loc[[c_c]].T,temp_all_significant_inverted.loc[[c_c]].T],axis=1)
    combined.columns = ['trend','not_trend','both','inverted']
    df_styled = combined.style.applymap(highlight_cells)
    #display(df_styled)
    #with HiddenPrints():
    dfi.export(df_styled, '..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_corr_styled.png'
"""

'\n\ncheck_corr_list = [\'^GSPC\',\'DGS3MO\',\'DGS2\',\'DGS10\']\n\nfor c_c in check_corr_list:\n\n    data = future_commodities_n_w_index_pvt_w_Fred[[c_c]]\n    dates = [pd.to_datetime(d).value for d in data.index]\n\n    dates = np.array(dates)-np.array(dates[0])/10**11\n\n    int_date_data = pd.concat([pd.DataFrame(dates),pd.DataFrame(data[[c_c]].values)],axis=1)\n    int_date_data.columns = [\'Date\',c_c]\n\n    windows = split_sequences(int_date_data.values,n_trend)\n    windows = pd.DataFrame(windows).head(-(n_trend-1)).values.ravel()        \n\n    trend_t = [*[trend_tau([pd.DataFrame(d).iloc[:,0].values,pd.DataFrame(d).iloc[:,1].values]) for d in windows]]\n    sp_w_trend = pd.concat([pd.DataFrame(data[c_c].values),pd.DataFrame(np.append(np.repeat(np.nan,len(data)-len([t[0] for t in trend_t])),[t[0] for t in trend_t])),pd.DataFrame(np.append(np.repeat(np.nan,len(data)-len([t[0] for t in trend_t])),[t[0] for t in trend_t]))],axis=1)\n    sp_w_trend.columns = [c_c,\'trending\',\'

In [50]:
#(pd.DataFrame(p_not_trending)<.05)
#(pd.DataFrame(p_trending)<.05)

In [51]:
"""
#test

dates = []
for d in pd.read_csv('..\\data\\raw\\sample.csv')['Date']:
    dates.append(dt.datetime.strptime(d, '%d-%b-%y'))

temp = pd.read_csv('..\\data\\raw\\sample.csv',thousands=",")
temp.index = dates
temp = temp.iloc[:,1:]
temp.columns = ['Open','High','Low','Adj Close','Volume']
#temp['Close'] = temp['Close'].astype(float)
temp.sort_index(inplace=True)
temp['Symbol'] = 'Test'
"""

'\n#test\n\ndates = []\nfor d in pd.read_csv(\'..\\data\\raw\\sample.csv\')[\'Date\']:\n    dates.append(dt.datetime.strptime(d, \'%d-%b-%y\'))\n\ntemp = pd.read_csv(\'..\\data\\raw\\sample.csv\',thousands=",")\ntemp.index = dates\ntemp = temp.iloc[:,1:]\ntemp.columns = [\'Open\',\'High\',\'Low\',\'Adj Close\',\'Volume\']\n#temp[\'Close\'] = temp[\'Close\'].astype(float)\ntemp.sort_index(inplace=True)\ntemp[\'Symbol\'] = \'Test\'\n'

In [52]:
"""

temp_new = derive_tvv(['temp',temp,nyse_dates]).sort_index(ascending=False)
temp_new.sort_index(inplace=True)
#display(temp_new)
#display(temp_new[['buy_volume_pct', 'sell_volume_pct', 'daily_buy_vol', 'daily_sell_vol', 'net_daily_vol', '20td_trend_vol', '20td_tot_daily_vol','20td_trend_vol_factor', '20td_tvf_rng_div_2', '20td_tvf_vel',  '50td_trend_vol', '50td_tot_daily_vol','50td_trend_vol_factor', '50td_tvf_rng_div_4', '50td_tvf_vel']])
#display(calculateMetrics(temp_new))
#
"""

"\n\ntemp_new = derive_tvv(['temp',temp,nyse_dates]).sort_index(ascending=False)\ntemp_new.sort_index(inplace=True)\n#display(temp_new)\n#display(temp_new[['buy_volume_pct', 'sell_volume_pct', 'daily_buy_vol', 'daily_sell_vol', 'net_daily_vol', '20td_trend_vol', '20td_tot_daily_vol','20td_trend_vol_factor', '20td_tvf_rng_div_2', '20td_tvf_vel',  '50td_trend_vol', '50td_tot_daily_vol','50td_trend_vol_factor', '50td_tvf_rng_div_4', '50td_tvf_vel']])\n#display(calculateMetrics(temp_new))\n#\n"

In [53]:
npa = []
for i in range(0,len([*SP1500,*indexes,*sectors])):
    npa.append([[*SP1500,*indexes,*sectors][i],start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
print("npa length:getStock(stocks)",len(npa))

# = getStock(npa[0])

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()
future = client.map(getStock, npa)
results_prices = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_prices.append("error")
    else:
        results_prices.append(f.result()) 
client.close()


npa length:getStock(stocks) 1520


In [54]:
npa = []
for i in range(0,len([*SP1500,*indexes,*sectors])):
    npa.append([[*SP1500,*indexes,*sectors][i]])
    
print("npa length:getTickerData",len(npa))

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()

future = client.map(getTickerData, npa)

results_ticker_data = []

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_ticker_data.append("error")
    else:
        results_ticker_data.append(f.result()) 
#client.restart()
client.close()


npa length:getTickerData 1520


In [55]:
print(results_ticker_data[0][1].info)
print(results_ticker_data[0][1].quarterly_balance_sheet)
print(results_ticker_data[0][1].quarterly_cashflow)
#print(results_ticker_data[0][1].quarterly_dividends)
print(results_ticker_data[0][1].quarterly_earnings)
print(results_ticker_data[0][1].quarterly_financials)


{'zip': '44124', 'sector': 'Basic Materials', 'fullTimeEmployees': 3443, 'longBusinessSummary': 'Materion Corporation, through with its subsidiaries, produces advanced engineered materials used in semiconductor, industrial, aerospace and defense, automotive, energy, consumer electronics, and telecom and data center in the United States, Asia, Europe, and internationally. It operates through Performance Alloys and Composites, Advanced Materials, and Precision Optics segments. The Performance Alloys and Composites segment offers advanced engineered solutions comprising beryllium and non-beryllium containing alloy systems and custom engineered parts in strip, bulk, rod, plate, bar, tube, and other customized shapes. This segment operates the bertrandite ore mine and refinery located in Utah that provides feedstock hydroxide for its beryllium businesses and external sale. The Advances Materials segment produces advanced chemicals, microelectronics packaging, precious metal, non-precious me

In [56]:
results_ticker_data[0][1].info

{'zip': '44124',
 'sector': 'Basic Materials',
 'fullTimeEmployees': 3443,
 'longBusinessSummary': 'Materion Corporation, through with its subsidiaries, produces advanced engineered materials used in semiconductor, industrial, aerospace and defense, automotive, energy, consumer electronics, and telecom and data center in the United States, Asia, Europe, and internationally. It operates through Performance Alloys and Composites, Advanced Materials, and Precision Optics segments. The Performance Alloys and Composites segment offers advanced engineered solutions comprising beryllium and non-beryllium containing alloy systems and custom engineered parts in strip, bulk, rod, plate, bar, tube, and other customized shapes. This segment operates the bertrandite ore mine and refinery located in Utah that provides feedstock hydroxide for its beryllium businesses and external sale. The Advances Materials segment produces advanced chemicals, microelectronics packaging, precious metal, non-precious

In [57]:
npa = []
results_ticker_names = [r[0] for r in results_ticker_data]
for n in range(0,len(results_ticker_names),1):
    npa.append([results_ticker_names[n],results_ticker_data[n][1]])

In [58]:

print("npa length:extractTickerValues",len(npa))  


npa length:extractTickerValues 1520


In [59]:
names_prices = [rp[0] for rp in results_prices]

#def process_df(data_):
npa = []    
for n in range(0,len(names_prices)):
    #print(names_prices[n])
    n_ = names_prices[n]
    npa.append([n_,results_prices[n][1],nyse_dates])
    
print("npa length:derive_tvv",len(npa))

prices_df = pd.DataFrame()

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()

future = client.map(derive_tvv, npa)
#results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        #results_sectors.append("error")
        pass
    else:
        #results_sectors.append(f.result())
        prices_df = pd.concat([f.result(),prices_df],axis=0)
client.close()

npa length:derive_tvv 1520


C:\Users\User\AppData\Local\Programs\3.9-JupyterLab\lib\site-packages\distributed\client.py:1131: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | 1.10.6 | None      | None    |
| lz4     | 4.0.2  | 3.1.10    | 3.1.10  |
| tornado | 6.2    | 6.1       | 6.1     |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [60]:
btc = get_historical_ohlc_data(symbol='BTCUSDT',past_days=delta.days,interval='1d')

btc_temp = btc.rename(columns={'symbol':'Symbol','open':'Open','high':'High','low':'Low','close':'Adj Close','volume':'Volume'})
btc_temp['Close'] = btc_temp['Adj Close']
btc_temp = btc_temp[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].astype(float)
btc_temp['Symbol']='btc'
btc_temp.index = pd.to_datetime([d.strftime('%Y-%m-%d') for d in btc['open_date_time']])

In [61]:

price_comm_df = pd.DataFrame()

for d in dict_comm_prices:
    temp = dict_comm_prices[d]
    temp['Symbol'] = d
    price_comm_df = pd.concat([price_comm_df,temp],axis=0)


price_comm_df = pd.concat([price_comm_df,pd.concat([prices_df[prices_df['Symbol']==s][['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol']] for s in sectors]),pd.concat([prices_df[prices_df['Symbol']==i][['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol']] for i in indexes]),btc_temp[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol']]],axis=0)

price_comm_df_pvt = pd.pivot_table(price_comm_df, values=['Adj Close'], index=price_comm_df.index,columns=['Symbol']).interpolate(limit_area='inside', method='time')
price_comm_df_pvt.columns = price_comm_df_pvt.columns.droplevel(0)

price_comm_df_pvt = price_comm_df_pvt.asfreq('D').interpolate(method='time',limit_area='inside')

In [62]:
#price_comm_df_10_yr_pvt.columns

In [63]:
#future_commodities_n_w_index_pvt_w_Fred = pd.concat([tbill_3mo_df,tbill_2y_df,tbill_10y_df,],axis=1)

price_comm_df_pvt = pd.pivot_table(price_comm_df, values=['Adj Close'], index=price_comm_df.index,columns=['Symbol']).interpolate(limit_area='inside', method='time')

#price_comm_df_pvt = pd.pivot_table(prices_comm_df_w_indexes_n_sector, values=['Adj Close'], index=prices_comm_df_w_indexes_n_sectors_pvt.index,columns=['Symbol']).interpolate(limit_area='inside', method='time')
price_comm_df_pvt.columns = price_comm_df_pvt.columns.droplevel(0)
#prices_comm_df_w_indexes_n_sectors_pvt = pd.pivot_table(prices_comm_df_w_indexes_n_sectors_pvt, values=['Adj Close'], index=prices_comm_df_w_indexes_n_sectors_pvt.index,columns=['Symbol']).interpolate(limit_area='inside', method='time')
#price_comm_df_pvt.columns = price_comm_df_pvt.columns.droplevel(0)
#price_comm_df_pvt.columns = [d for d in dict_comm_prices]

#9 quarters * 5 = 11.25 years * .9 ~ 10.25 years
price_comm_df_10_yr_pvt = price_comm_df_pvt.dropna(axis=1, thresh = int(0.90*price_comm_df_pvt.shape[0]))
#price_comm_df_10_yr_pvt.columns = price_comm_df_10_yr_pvt.columns.droplevel(0)
price_comm_df_5_yr_pvt = price_comm_df_pvt.tail(int(len(price_comm_df_10_yr_pvt)/2))
#price_comm_df_5_yr_pvt.columns = price_comm_df_5_yr_pvt.columns.droplevel(0)
price_comm_df_5_yr_pvt = price_comm_df_5_yr_pvt.dropna(axis=1, thresh = int(0.95*price_comm_df_5_yr_pvt.shape[0]))
price_comm_df_2_yr_pvt = price_comm_df_pvt.tail(int(len(price_comm_df_10_yr_pvt)/5))
#price_comm_df_2_yr_pvt.columns = price_comm_df_2_yr_pvt.columns.droplevel(0)
price_comm_df_2_yr_pvt = price_comm_df_2_yr_pvt.dropna(axis=1, thresh = int(0.95*price_comm_df_2_yr_pvt.shape[0]))

price_comm_df_10_yr_pvt_corr = price_comm_df_10_yr_pvt.asfreq('D').interpolate(method='time').asfreq('W-'+price_comm_df_10_yr_pvt.index[-1].strftime('%a')).corr()

price_comm_df_5_yr_pvt_corr = price_comm_df_5_yr_pvt.asfreq('D').interpolate(method='time').asfreq('W-'+price_comm_df_5_yr_pvt.index[-1].strftime('%a')).corr()

price_comm_df_2_yr_pvt_corr = price_comm_df_2_yr_pvt.asfreq('D').interpolate(method='time').asfreq('W-'+price_comm_df_2_yr_pvt.index[-1].strftime('%a')).corr()

writer = pd.ExcelWriter('..\\reports\\'+end.strftime('%Y-%m-%d')+'_correlations.xlsx', engine='xlsxwriter')
writer = pd.ExcelWriter('..\\reports\\_correlations.xlsx', engine='xlsxwriter')


merged_dict = dict_treasury | dict_currencies_futures | dict_energy | dict_foods | dict_indexes_w_futures | dict_materials_futures | dict_metals

templist = [merged_dict[d] for d in merged_dict]

A = templist

#price_comm_df_10_yr_pvt_corr[price_comm_df_10_yr_pvt_corr.columns.intersection([*[dict_treasury[d] for d in dict_treasury], *[dict_metals[d] for d in dict_metals], *[dict_currencies_futures[d] for d in dict_currencies_futures], *[dict_energy[d] for d in dict_energy],*[dict_foods[d] for d in dict_foods], *[dict_indexes_w_futures[d] for d in dict_indexes_w_futures], *[dict_materials_futures[d] for d in dict_materials_futures]])].corr().to_excel(writer, sheet_name='price_comm_curr_10_yr_corr')


B = price_comm_df_10_yr_pvt_corr.columns
common = sorted(set(A).intersection(B) ,key=lambda x:A.index(x))

reordered_10_yr = price_comm_df_10_yr_pvt_corr[common]
reordered_10_yr.columns = [dict((v, k) for k, v in merged_dict.items())[f] for f in common]
reordered_10_yr.loc[common].T.to_excel(writer, sheet_name='prc_comm_10_yr_wt_corr')

#price_comm_df_5_yr_pvt_corr[price_comm_df_5_yr_pvt_corr.columns.intersection([*[dict_treasury[d] for d in dict_treasury], *[dict_metals[d] for d in dict_metals], *[dict_currencies_futures[d] for d in dict_currencies_futures], *[dict_energy[d] for d in dict_energy],*[dict_foods[d] for d in dict_foods], *[dict_indexes_w_futures[d] for d in dict_indexes_w_futures], *[dict_materials_futures[d] for d in dict_materials_futures]])].corr().to_excel(writer, sheet_name='price_comm_curr_5_yr_corr')


B = price_comm_df_5_yr_pvt_corr.columns
common = sorted(set(A).intersection(B) ,key=lambda x:A.index(x))

reordered_5_yr = price_comm_df_5_yr_pvt_corr[common]
reordered_5_yr.columns = [dict((v, k) for k, v in merged_dict.items())[f] for f in common]
reordered_5_yr.loc[common].T.to_excel(writer, sheet_name='prc_comm_5_yr_wt_corr')

#price_comm_df_2_yr_pvt_corr[price_comm_df_2_yr_pvt_corr.columns.intersection([*[dict_treasury[d] for d in dict_treasury], *[dict_metals[d] for d in dict_metals], *[dict_currencies_futures[d] for d in dict_currencies_futures], *[dict_energy[d] for d in dict_energy],*[dict_foods[d] for d in dict_foods], *[dict_indexes_w_futures[d] for d in dict_indexes_w_futures], *[dict_materials_futures[d] for d in dict_materials_futures]])].corr().to_excel(writer, sheet_name='price_comm_curr_2_yr_corr')


B = price_comm_df_2_yr_pvt_corr.columns
common = sorted(set(A).intersection(B) ,key=lambda x:A.index(x))

reordered_2_yr = price_comm_df_2_yr_pvt_corr[common]
reordered_2_yr.columns = [dict((v, k) for k, v in merged_dict.items())[f] for f in common]
reordered_2_yr.loc[common].T.to_excel(writer, sheet_name='prc_comm_2_yr_wt_corr')

name_set = [*[dict_treasury[d] for d in dict_treasury], *[dict_metals[d] for d in dict_metals], *[dict_currencies_futures[d] for d in dict_currencies_futures], *[dict_energy[d] for d in dict_energy],*[dict_foods[d] for d in dict_foods], *[dict_indexes_w_futures[d] for d in dict_indexes_w_futures], *[dict_materials_futures[d] for d in dict_materials_futures], *[s for s in sectors],*[i for i in indexes],'btc']

a = price_comm_df_10_yr_pvt_corr[price_comm_df_10_yr_pvt_corr.columns.intersection(name_set)].corr()
b = price_comm_df_5_yr_pvt_corr[price_comm_df_5_yr_pvt_corr.columns.intersection(name_set)].corr()
c = price_comm_df_2_yr_pvt_corr[price_comm_df_2_yr_pvt_corr.columns.intersection(name_set)].corr()

weighted_10y_corr = ((a+b+c)/4)

A = templist
B = weighted_10y_corr.columns
common = sorted(set(A).intersection(B) ,key=lambda x:A.index(x))

reordered_10_yr = weighted_10y_corr[common]
reordered_10_yr.columns = [dict((v, k) for k, v in merged_dict.items())[f] for f in common]
reordered_10_yr.loc[common].T.to_excel(writer, sheet_name='prc_comm_wgt_10_yr_wt_corr')
writer.close()

future_commodities_n_w_index_pvt_w_Fred = price_comm_df_10_yr_pvt[price_comm_df_10_yr_pvt.columns.intersection(name_set)]

future_commodities_n_w_index_pvt_w_Fred.to_csv('..\\data\\processed\\'+end.strftime('%Y-%m-%d')+'_future_commodities_n_w_index_pvt_w_Fred.csv')

future_commodities_n_w_index_pvt_w_Fred.dropna(axis=1, thresh = int(.95*future_commodities_n_w_index_pvt_w_Fred.shape[0]), inplace=True)


In [64]:
temp_df_h = future_commodities_n_w_index_pvt_w_Fred.dropna().pct_change().corr().style.applymap(highlight_cells,threshold=critical_95t2(len(future_commodities_n_w_index_pvt_w_Fred.dropna()))[1])
dfi.export(temp_df_h, '..\\data\\processed\\'+end.strftime('%Y-%m-%d')+'-weekly_stocks_w_fred_n_commodities_corr_styled.png',max_cols=-1)
dfi.export(temp_df_h, '..\\data\\processed\\weekly_stocks_w_fred_n_commodities_corr_styled.png',max_cols=-1)


In [65]:
price_comm_df_pvt.to_csv('..\\data\\processed\\'+end.strftime('%Y-%m-%d')+'_price_comm_df_pvt.csv')
price_comm_df_pvt.to_csv('..\\data\\processed\\_price_comm_df_pvt.csv')

In [66]:

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()

future = client.map(extractTickerValues, npa)

results_ticker_values = []

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_ticker_values.append("error")
    else:
        results_ticker_values.append(f.result()) 
client.close()

"""
trackers = ['beta','bookValue','currentRatio','debtToEquity','dividendRate','dividendYield','earningsGrowth','ebitda','ebitdaMargins','enterpriseValue','enterpriseToEbitda','enterpriseToRevenue','floatShares','forwardPE','freeCashflow','grossMargins','grossProfits','heldPercentInsiders','heldPercentInstitutions','impliedSharesOutstanding','industry','marketCap','operatingCashflow','operatingMargins','payoutRatio','pegRatio','priceToBook','priceToSalesTrailing12Months','profitMargins','quickRatio','returnOnAssets','returnOnEquity','revenueGrowth','revenuePerShare','sector','sharesOutstanding','sharesShort','sharesPercentSharesOut','sharesShortPreviousMonthDate','shortRatio','totalAssets','totalCash','totalCashPerShare','totalDebt','totalRevenue','trailingPE']

for t in trackers:
    if(np.sum(np.where(subset.columns==t))==0):
        values = pd.concat([values,pd.DataFrame('error',index=[n_],columns=[t])],axis=1)
    else:
        values = pd.concat([values,pd.DataFrame(subset[t].values[0],index=[n_],columns=[t])],axis=1)
"""

C:\Users\User\AppData\Local\Programs\3.9-JupyterLab\lib\site-packages\distributed\client.py:1131: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | 1.10.6 | None      | None    |
| lz4     | 4.0.2  | 3.1.10    | 3.1.10  |
| tornado | 6.2    | 6.1       | 6.1     |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


"\ntrackers = ['beta','bookValue','currentRatio','debtToEquity','dividendRate','dividendYield','earningsGrowth','ebitda','ebitdaMargins','enterpriseValue','enterpriseToEbitda','enterpriseToRevenue','floatShares','forwardPE','freeCashflow','grossMargins','grossProfits','heldPercentInsiders','heldPercentInstitutions','impliedSharesOutstanding','industry','marketCap','operatingCashflow','operatingMargins','payoutRatio','pegRatio','priceToBook','priceToSalesTrailing12Months','profitMargins','quickRatio','returnOnAssets','returnOnEquity','revenueGrowth','revenuePerShare','sector','sharesOutstanding','sharesShort','sharesPercentSharesOut','sharesShortPreviousMonthDate','shortRatio','totalAssets','totalCash','totalCashPerShare','totalDebt','totalRevenue','trailingPE']\n\nfor t in trackers:\n    if(np.sum(np.where(subset.columns==t))==0):\n        values = pd.concat([values,pd.DataFrame('error',index=[n_],columns=[t])],axis=1)\n    else:\n        values = pd.concat([values,pd.DataFrame(subset[

In [67]:
 
ticker_values = pd.DataFrame()

pickle_dumps = []

for r in results_ticker_values:
    if(r=='error'):
        pass
        #pd.DataFrame(np.repeat('error', len(trackers), axis = None),index=)
    else:                
        #print(r[1])
        subset = pd.DataFrame(r[1])
        subset.index = [r[0]]
        #print(subset)
        ticker_values = pd.concat([ticker_values,subset],axis=0)
        pickle_dumps.append([r[0],r[2]])
        #ticker_values = pd.concat([ticker_values,pd.DataFrame(r[1],index=[r[0]])],axis=0)

pd.DataFrame(ticker_values)

pickle.dump(pickle_dumps, open('..\\data\\interim\\fundamental.pkl', 'wb'))

In [68]:
"""
missing_ticker_values = list(set([*SP1500,*indexes,*sectors]).difference([n for n in pd.DataFrame(ticker_values).index]))
print(missing_ticker_values)

for r in range(0,len(missing_ticker_values)):
    r_ = missing_ticker_values[r]
    temp = pd.DataFrame(np.array([r_,*np.repeat('missing',len(pd.DataFrame(ticker_values.columns)))])).T.set_index(0)
    temp.columns = ticker_values.columns
    #.set_index(0).iloc[[0]].columns))])
    ticker_values = pd.concat([ticker_values,temp],axis=0)
"""


"\nmissing_ticker_values = list(set([*SP1500,*indexes,*sectors]).difference([n for n in pd.DataFrame(ticker_values).index]))\nprint(missing_ticker_values)\n\nfor r in range(0,len(missing_ticker_values)):\n    r_ = missing_ticker_values[r]\n    temp = pd.DataFrame(np.array([r_,*np.repeat('missing',len(pd.DataFrame(ticker_values.columns)))])).T.set_index(0)\n    temp.columns = ticker_values.columns\n    #.set_index(0).iloc[[0]].columns))])\n    ticker_values = pd.concat([ticker_values,temp],axis=0)\n"

In [69]:
market_data = pd.concat([ticker_values])

In [70]:

npa = []
for s in names_prices:
    #print(s)
    subset = prices_df.iloc[np.where(prices_df['Symbol']==s)]
    if(len(subset)==0):
        pass
    else:
        npa.append(subset)
        
print("npa length:calculateMetrics",len(npa))

screenerMetrics = []

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()

future = client.map(calculateMetrics, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        screenerMetrics.append("error")
    else:
        screenerMetrics.append(f.result()) 
client.close()



npa length:calculateMetrics 1519


In [71]:
screener_metrics = pd.concat(screenerMetrics)

In [72]:

missing_screener_metrics = list(set([*SP1500,*indexes,*sectors]).difference([n for n in screener_metrics.index]))
print(missing_screener_metrics)

for r in range(0,len(missing_screener_metrics)):
    r_ = missing_screener_metrics[r]
    test = pd.DataFrame([np.repeat('missing',len(screener_metrics.columns))],index=[r_],columns=screener_metrics.columns)
    screener_metrics = pd.concat([screener_metrics,test],axis=0)

#pd.DataFrame(screener_metrics)

['JW-A']


In [73]:
#fundamentals_quarterlies = pickle.load(open('./fundamental.pkl', 'rb'))
fundamental_entries = [e[0] for e in pickle_dumps]#[e[0] for e in fundamentals_quarterlies]


In [74]:
#summary = pd.concat([prices_df[prices_df['Symbol']==name_][['Symbol','MIN_Lookback_One_Year', 'MIN_Lookback_2_Months', 'MIN_Lookback_1_Months','risk_trend_factor','risk_per_share','volume_factor']].iloc[[-1]].reset_index().set_index('Symbol') for name_ in np.unique(prices_df['Symbol'].values)]).reset_index()

In [75]:

#threshold = 0 
#stock_fundamentals = market_data
#screener = screener_metrics
#stock_indexes = sp1500_index_df


#spdr_indexes = indexes
#spdr_indexes.columns=['Symbol']
#vanguard_sectors = sectors
#vanguard_sectors.columns=['Symbol']
screener_metrics_w_sp1500index = screener_metrics.reset_index().rename(columns={'index': 'Symbol'}).merge(sp1500_index_df,on='Symbol',how='left')

screener_metrics_w_sp1500index_good = screener_metrics_w_sp1500index[screener_metrics_w_sp1500index['50td_tvf_vel']!='missing']

#risk_trend_threshold = screener_metrics_w_sp1500index_good['50td_tvf_vel'].quantile(q=[threshold]).values[0]
stock_sectors = market_data['sector']

temp = stock_sectors.reset_index()
temp.columns = ['Symbol','Sector']
sector_merged = screener_metrics.reset_index().rename(columns={'index': 'Symbol'}).merge(temp, on='Symbol', how='right')
temp_sm = sector_merged.merge(sp1500_index_df, on='Symbol', how='left').set_index('Symbol')
#temp_sm.columns = ['risk_trend_factor', 'Sector', 'index']

temp_index_symbol_map = pd.concat([sp1500_index_df,pd.DataFrame([invert_dict_indexes[s] for s in sp1500_index_df['Market Index']])],axis=1)
temp_index_symbol_map.columns = ['Symbol','Market Index','Index Symbol']

index_sector_merged = temp_sm.merge(temp_index_symbol_map.iloc[:,[0,2]], on='Symbol', how='left')

"""
for spdr_ in range(0,len(indexes)):
    spdr__ = indexes[spdr_]
    #spdr__i = indexes[spdr_]
    #temp_sm.iloc[np.where(temp_sm.index==spdr__)][['index']] = spdr__i
    temp_sm.iloc[np.where(temp_sm.index==spdr__)][['index_Symbol']] = spdr__
    """

"\nfor spdr_ in range(0,len(indexes)):\n    spdr__ = indexes[spdr_]\n    #spdr__i = indexes[spdr_]\n    #temp_sm.iloc[np.where(temp_sm.index==spdr__)][['index']] = spdr__i\n    temp_sm.iloc[np.where(temp_sm.index==spdr__)][['index_Symbol']] = spdr__\n    "

In [76]:


sector_performance = []
for v in sectors:    
    #sector_performance.append([v,screener_metrics.loc[[v]]['risk_trend_factor'].values[0]])
    sector_performance.append([v,np.mean(index_sector_merged[index_sector_merged['Sector']==dict_sectors_reverse[v]]['50td_tvf_vel']),np.mean(index_sector_merged[index_sector_merged['Sector']==dict_sectors_reverse[v]]['20td_tvf_vel']),np.mean(index_sector_merged[index_sector_merged['Sector']==dict_sectors_reverse[v]]['risk_trend_factor'])])
temp_v = pd.concat([pd.DataFrame([dict_sectors_reverse[v] for v in sectors]),pd.DataFrame(sectors)],axis=1)
temp_v.columns = ['Sector','Symbol']
sectors_risk_sorted = pd.DataFrame(sector_performance,columns=['Symbol','50td_tvf_vel','20td_tvf_vel','risk_trend_factor']).merge(temp_v, on='Symbol', how='left').sort_values(by='50td_tvf_vel',ascending=False)    
sectors_risk_sorted

,Symbol,50td_tvf_vel,20td_tvf_vel,risk_trend_factor,Sector
0,VOX,NaN,NaN,NaN,Communication Services
1,VCR,NaN,NaN,NaN,Consumer Cyclical
2,VDC,NaN,NaN,NaN,Consumer Defensive
3,VDE,NaN,NaN,NaN,Energy
4,VFH,NaN,NaN,NaN,Financial Services
5,VHT,NaN,NaN,NaN,Healthcare
6,VIS,NaN,NaN,NaN,Industrials
7,VGT,NaN,NaN,NaN,Technology
8,VAW,NaN,NaN,NaN,Basic Materials
9,VNQ,NaN,NaN,NaN,Real Estate


In [77]:
index_performance = []
for v in indexes:
    #index_performance.append([v,screener_metrics.loc[[v]]['20td_tvf_vel'].values[0]])
    if(v =='SPTM'):
        index_performance.append([v,np.nanmean(index_sector_merged['50td_tvf_vel'].replace(['error','missing'], np.nan))])
    else:
        index_performance.append([v,np.nanmean(index_sector_merged[index_sector_merged['Market Index']==dict_indexes[v]]['50td_tvf_vel'].replace(['error','missing'], np.nan))])
    
    
temp_i = pd.DataFrame(index_performance,index=indexes)
temp_i.columns = ['Symbol','50td_tvf_vel']
index_risk_sorted = pd.DataFrame(index_performance,columns=['Symbol','50td_tvf_vel'],index=[dict_indexes[i] for i in indexes]).sort_values(by='50td_tvf_vel',ascending=False).reset_index()
index_risk_sorted.columns = ['Index','Symbol','50td_tvf_vel']
index_risk_sorted

,Index,Symbol,50td_tvf_vel
0,SP400,MDYG,1.423889
1,SP600,SLY,0.983610
2,SP1500,SPTM,0.921024
3,SP500,SPY,0.404664


In [78]:
risk_screened = screener_metrics_w_sp1500index_good[(screener_metrics_w_sp1500index_good['volume_factor']!= 'missing')]
risk_screened = risk_screened.merge(temp, on='Symbol', how='left')
print(risk_screened.columns)
#temp_df = risk_screened[['Symbol','Adj Close','MIN_Lookback_One_Year', 'MIN_Lookback_2_Months', 'MIN_Lookback_1_Months','risk_trend_factor','volume_factor','adf','hurst','Sector','Market Index']]
#temp_df.set_index('Symbol',inplace=True)
#pd.concat([temp_df,temp_sm],axis=0)

#screened_final_set = risk_screened

Index(['Symbol', 'Date', 'buy_volume_pct', 'sell_volume_pct', 'daily_buy_vol',
       'daily_sell_vol', 'net_daily_vol', '20td_trend_vol',
       '20td_tot_daily_vol', '20td_trend_vol_factor', '20td_tvf_rng_div_2',
       '20td_tvf_vel', '50td_trend_vol', '50td_tot_daily_vol',
       '50td_trend_vol_factor', '50td_tvf_rng_div_4', '50td_tvf_vel',
       'MIN_Lookback_One_Year', 'MIN_Lookback_2_Months',
       'MIN_Lookback_1_Months', 'risk_trend_factor', 'risk_per_share',
       'volume_factor', 'Adj Close', '50tdBOLD', '50tdMA-TP', '50tdBOLU',
       '200tdSMA', '200tdSDev', 'adf_50td', 'tau_50td', 'tau_50td_p',
       'hurst_50td', 'Market Index', 'Sector'],
      dtype='object')


In [79]:
risk_screened
#risk_screened['20td_tvf_vel']

,Symbol,Date,buy_volume_pct,sell_volume_pct,daily_buy_vol,daily_sell_vol,net_daily_vol,20td_trend_vol,20td_tot_daily_vol,20td_trend_vol_factor,...,50tdMA-TP,50tdBOLU,200tdSMA,200tdSDev,adf_50td,tau_50td,tau_50td_p,hurst_50td,Market Index,Sector
0,UGI,2022-08-25 00:00:00,0.511905,-0.488095,307859.686377,-293540.313623,14319.372755,-2080197.690456,22156100.0,-0.093888,...,40.430933,43.18214,40.156003,3.423561,0.121973,0.565591,0.0,NaN,SP400,error
1,SCSC,2022-08-25 00:00:00,0.851563,-0.148437,346501.160206,-60398.839794,286102.320411,97619.762399,3366400.0,0.028998,...,31.885333,34.602779,33.59345,2.303571,0.053434,0.208333,0.032909,NaN,SP600,error
2,TECH,2022-08-25 00:00:00,0.924954,-0.075046,160849.54696,-13050.45304,147799.09392,422683.88417,3887800.0,0.108721,...,358.824801,393.568326,404.9446,49.072282,0.083691,0.368163,0.000162,NaN,SP500,error
3,STE,2022-08-25 00:00:00,0.971301,-0.028699,322860.566293,-9539.433707,313321.132586,575913.429263,9347300.0,0.061613,...,209.89,223.741719,225.422557,12.554112,0.163946,0.338776,0.000518,NaN,SP500,error
4,HLX,2022-08-25 00:00:00,0.874997,-0.125003,1315208.580369,-187891.419631,1127317.160739,3152455.205035,32268200.0,0.097695,...,3.4678,4.612835,3.91735,0.676371,0.845899,0.415884,0.000021,NaN,SP600,error
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514,HAE,2022-08-25 00:00:00,0.770641,-0.229359,253463.966739,-75436.033261,178027.933477,309612.607058,9010600.0,0.034361,...,68.0588,77.754041,58.81375,7.176181,0.900929,0.754703,0.0,NaN,SP400,error
1515,BC,2022-08-25 00:00:00,0.611354,-0.388646,240751.25785,-153048.74215,87702.5157,2615493.977112,12437600.0,0.210289,...,74.429067,88.074238,84.382103,10.984992,0.796305,0.835102,0.0,NaN,SP400,error
1516,FCFS,2022-08-25 00:00:00,0.844721,-0.155279,119865.846862,-22034.153138,97831.693725,105666.288082,3102600.0,0.034057,...,71.526333,81.290726,71.08205,4.634761,0.855199,0.499796,0.0,NaN,SP400,error
1517,ASB,2022-08-25 00:00:00,0.926828,-0.073172,740349.859973,-58450.140027,681899.719945,5590128.663228,23273000.0,0.240198,...,19.588533,22.003672,21.783405,2.031099,0.890493,0.679478,0.0,NaN,SP400,error


In [80]:
#threshold_value = screener_metrics_w_sp1500index_good.dropna()['50td_tvf_vel'].quantile([threshold])

In [81]:
_50td_tvf_vel_sector_dict = dict()
_20td_tvf_vel_sector_dict = dict()
_risk_trend_factor_sector_dict = dict()
for s in sectors:
    dict_sectors_reverse[s]
    #_20td_tvf_vel_sector_dict[s] = screener_metrics.loc[[s]]['20td_tvf_vel'].values[-1]
    _50td_tvf_vel_sector_dict[s] = np.mean(index_sector_merged[index_sector_merged['Sector']==dict_sectors_reverse[s]]['50td_tvf_vel'])
    _20td_tvf_vel_sector_dict[s] = np.mean(index_sector_merged[index_sector_merged['Sector']==dict_sectors_reverse[s]]['20td_tvf_vel'])
    _risk_trend_factor_sector_dict[s] = np.mean(index_sector_merged[index_sector_merged['Sector']==dict_sectors_reverse[s]]['risk_trend_factor'])
    
_50td_tvf_vel_sector_dict['error']='error'
_20td_tvf_vel_sector_dict['error']='error'
_risk_trend_factor_sector_dict['error']='error'

In [82]:
"""
inv_dict_map = {v: k for k, v in dict_indexes.items()}
inv_dict_map[np.nan] = 'error'
in_ = [inv_dict_map[t[0]] for t in temp_sm[['Market Index']].values]
se_ = [dict_sectors[t[0]] for t in temp_sm[['Sector']].values]
#temp_sm[['Sector','index']]
"""
#symbols_w_index_sector = pd.concat([pd.DataFrame(se_,columns=['index'],index=temp_sm.index),pd.DataFrame(in_,columns=['sector'],index=temp_sm.index)],axis=1)


"\ninv_dict_map = {v: k for k, v in dict_indexes.items()}\ninv_dict_map[np.nan] = 'error'\nin_ = [inv_dict_map[t[0]] for t in temp_sm[['Market Index']].values]\nse_ = [dict_sectors[t[0]] for t in temp_sm[['Sector']].values]\n#temp_sm[['Sector','index']]\n"

In [83]:
#symbols_w_index_sector

In [84]:
#pd.DataFrame([dict_sectors[s] for s in index_sector_merged['Sector']],columns=['Sector Symbol'],index=index_sector_merged['Symbol']
sector_symbols = pd.DataFrame([dict_sectors[s] for s in index_sector_merged['Sector']],index=index_sector_merged['Symbol'],columns=['Sector Symbol'])

a = pd.DataFrame([_50td_tvf_vel_sector_dict[s] for s in sector_symbols['Sector Symbol']],columns=['sector_50td_tvf_vel'],index=sector_symbols.index)
b = pd.DataFrame([_20td_tvf_vel_sector_dict[s] for s in sector_symbols['Sector Symbol']],columns=['sector_20td_tvf_vel'],index=sector_symbols.index)
c = pd.DataFrame([_risk_trend_factor_sector_dict[s] for s in sector_symbols['Sector Symbol']],columns=['sector_risk_trend_factor'],index=sector_symbols.index)

#screener_final = pd.concat([screener_metrics,symbols_w_index_sector.set_index),symbols_w_sector_rtf,index_sector_merged[['Market Index','Sector']]],axis=1)

symbols_w_sector_rtf = pd.concat([a,b,c],axis=1)

In [85]:
screener_final = pd.concat([screener_metrics,symbols_w_sector_rtf,index_sector_merged.set_index('Symbol')[['Sector']],sector_symbols,index_sector_merged.set_index('Symbol')[['Market Index','Index Symbol']],market_data.loc[:, ~market_data.columns.isin(["sector"])]],axis=1)

a = screener_final[screener_final['50td_tvf_vel'].apply(type) != np.str_].sort_values(by='50td_tvf_vel',ascending=False)
b = screener_final[screener_final['50td_tvf_vel'].apply(type) == np.str_]
screener_sorted = pd.concat([a,b],axis=0)

In [86]:
#pd.concat([screener_metrics.sort_values(by='risk_trend_factor',ascending=False).iloc[:,1:],symbols_w_index_sector,symbols_w_sector_rtf,temp_sm[['Market Index','Sector']]],axis=1)

In [87]:
list_sector_n_indexes = []
list_stocks = []
for i in index_risk_sorted['Symbol'].values:
    list_sector_n_indexes.append(i)
for i in sectors_risk_sorted['Symbol'].values:
    list_sector_n_indexes.append(i)
#for i in screened_final_set['Symbol'].values:
for i in sp1500_index_df['Symbol'].values:
    list_stocks.append(i)
stocks_ = list_stocks
#stocks_ = np.unique(list_stocks, return_index=True)[1]
#list__ = [list_[symbol] for symbol in sorted(stocks_)]
#list__ = ['MCK','MPC','APA','LNTH','MUSA','CIVI','VAW','VGT','EQT','XOM','DVN','MRO','EOG','HES','CEIX','SM','RRC','MUR','MTDR','PXD','OXY','PDCE','NFG','FANG','TWI','PBF','MANT','HRB','VDC','ACC','GO','TVTY','FCN','VIVO']

In [88]:
#screener_sorted.head(50)

In [89]:
screener_sorted.to_excel(writer, sheet_name='screener_sorted')


screener_sorted.to_csv('..\\data\\processed\\'+end.strftime('%Y-%m-%d')+'_screener_sorted.csv')
screener_sorted.to_csv('..\\data\\processed\\screener_sorted.csv')



In [90]:
#screener_final.loc[np.setdiff1d([*screener_final.index],list_sector_n_indexes)]['risk_trend_factor']

In [91]:
writer.close()

C:\Users\User\AppData\Local\Programs\3.9-JupyterLab\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [92]:
npa = []
for f in FRED_Indicators:
    npa.append([f,start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d'),nyse_dates])

In [93]:
completed_fred = pd.DataFrame()

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()

future = client.map(Fred_Data, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        pass
    else:
        completed_fred = pd.concat([completed_fred,f.result()],axis=0)
client.close()

completed_fred_pvt = pd.pivot_table(completed_fred, values=['Value'], index=completed_fred.index,columns=['Symbol'])
completed_fred_pvt

C:\Users\User\AppData\Local\Programs\3.9-JupyterLab\lib\site-packages\distributed\client.py:1131: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | 1.10.6 | None      | None    |
| lz4     | 4.0.2  | 3.1.10    | 3.1.10  |
| tornado | 6.2    | 6.1       | 6.1     |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Value                                                   \
Symbol     ASPUS AWHAETP B432RG3A086NBEA BAA10Y BACDINA066MNFRBNY   
2017-04-26   NaN     NaN             NaN   2.27               NaN   
2017-04-27   NaN     NaN             NaN   2.28               NaN   
2017-04-28   NaN     NaN             NaN   2.28               NaN   
2017-05-01   NaN    34.4             NaN   2.29          6.600000   
2017-05-02   NaN    34.4             NaN   2.30          6.496774   
...          ...     ...             ...    ...               ...   
2022-08-22   NaN     NaN             NaN   2.24               NaN   
2022-08-23   NaN     NaN             NaN   2.24               NaN   
2022-08-24   NaN     NaN             NaN   2.23               NaN   
2022-08-25   NaN     NaN             NaN   2.21               NaN   
2022-08-26   NaN     NaN             NaN    NaN               NaN   

                                                                       \
Symbol     BACTSAMFRBDAL BAMLC0A0CM BAMLCC0A1AAATRIV BAMLCC0A4BBBTRIV   
2017-04-26           NaN       1.23           603.12           783.00   
2017-04-27           NaN       1.22           603.70           783.83   
2017-04-28           NaN       1.22           604.37           784.68   
2017-05-01     18.800000       1.23           601.41           782.79   
2017-05-02     18.667742       1.22           603.99           784.85   
...                  ...        ...              ...              ...   
2022-08-22           NaN       1.44           664.48           869.43   
2022-08-23           NaN       1.45           664.42           868.91   
2022-08-24           NaN       1.43           660.66           866.54   
2022-08-25           NaN       1.42           667.16           872.12   
2022-08-26           NaN        NaN              NaN              NaN   

                        ...                                                  \
Symbol     BAMLH0A3HYC  ... VIXCLS VXVCLS  WDFUELLA    WGS3MO WILLLRGCAPVAL   
2017-04-26        8.42  ...  10.85  12.98       NaN       NaN      71731.05   
2017-04-27        8.52  ...  10.36  12.80       NaN       NaN      71681.86   
2017-04-28        8.44  ...  10.82  12.93  1.577000  0.810000      71418.58   
2017-05-01        8.54  ...  10.11  12.63  1.542286  0.827143      71376.20   
2017-05-02        8.43  ...  10.59  12.83  1.530714  0.832857      71333.22   
...                ...  ...    ...    ...       ...       ...           ...   
2022-08-22       10.09  ...  23.80  26.00       NaN       NaN     121139.85   
2022-08-23       10.16  ...  24.11  25.92       NaN       NaN     120991.41   
2022-08-24       10.04  ...  22.82  24.89       NaN       NaN     121395.52   
2022-08-25       10.05  ...  21.78  24.13       NaN       NaN     122818.67   
2022-08-26         NaN  ...    NaN    NaN       NaN       NaN           NaN   

                                                                        
Symbol     WILLMICROCAPPR    WPU0911      WPU101 WPUSI019011    WTB3MS  
2017-04-26       12866.05        NaN         NaN         NaN       NaN  
2017-04-27       12817.95        NaN         NaN         NaN       NaN  
2017-04-28       12697.00        NaN         NaN         NaN  0.800000  
2017-05-01       12708.58  172.90000  213.700000  353.800000  0.817143  
2017-05-02       12638.25  173.03871  213.706452  353.793548  0.822857  
...                   ...        ...         ...         ...       ...  
2022-08-22       14231.52        NaN         NaN         NaN       NaN  
2022-08-23       14339.47        NaN         NaN         NaN       NaN  
2022-08-24       14500.93        NaN         NaN         NaN       NaN  
2022-08-25       14622.61        NaN         NaN         NaN       NaN  
2022-08-26            NaN        NaN         NaN         NaN       NaN  

[1345 rows x 145 columns]

In [94]:
npa = []
for e in etf_bonds:
    npa.append([e,start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d'),nyse_dates])
    
completed_bonds = pd.DataFrame()

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()

future = client.map(dl, npa)

#my intent was to capture future objects vs results and this gave me resultsa
for f in as_completed(future):
    if(f.status=="error"):
        pass
    else:
        completed_bonds = pd.concat([completed_bonds,f.result()],axis=0)
client.close()

completed_bonds_pvt = pd.pivot_table(completed_bonds, values='Close', index=completed_bonds.index,columns=['Symbol'])
completed_bonds_pvt

C:\Users\User\AppData\Local\Programs\3.9-JupyterLab\lib\site-packages\distributed\client.py:1131: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | 1.10.6 | None      | None    |
| lz4     | 4.0.2  | 3.1.10    | 3.1.10  |
| tornado | 6.2    | 6.1       | 6.1     |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Symbol,AGG,AGZ,BIL,GBF,GOVT,IEF,IEI,ILTB,IMTB,ISTB,IUSB,LQD,MEAR,MUB,NEAR,SHV,SHY,SUB,TLT
2017-04-26,96.140221,103.088181,87.133598,101.800583,22.990881,97.705383,114.566322,51.243267,42.689381,44.672123,44.326092,106.138618,47.743431,97.281525,45.930504,104.415352,79.413300,99.560692,109.352325
2017-04-27,96.228447,103.179016,87.114555,101.764702,23.009144,97.769691,114.640442,51.193817,42.776630,44.698792,44.361000,106.236961,47.674812,97.334908,45.930504,104.424782,79.441498,99.551308,109.316521
2017-04-28,96.396103,103.206322,87.152641,101.791595,23.009144,97.870674,114.668236,51.424603,42.863022,44.707680,44.387180,106.362083,47.674812,97.299301,45.939640,104.415352,79.479080,99.475990,109.558266
2017-05-01,96.147652,103.103462,87.146942,101.667671,22.975323,97.659187,114.582870,51.020256,42.809315,44.670280,44.318092,106.032288,47.647160,97.123657,45.931396,104.425766,79.424538,99.531578,108.647507
2017-05-02,96.280251,103.194481,87.127892,101.892174,23.011892,97.861443,114.731369,51.317951,42.862144,44.723709,44.396809,106.345978,47.752068,97.266342,45.949711,104.416290,79.452744,99.531578,109.203850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-19,101.989998,109.800003,91.489998,107.610001,23.809999,102.480003,119.059998,55.770000,45.154999,47.430000,47.080002,111.599998,49.730000,106.879997,49.349998,110.089996,82.529999,104.370003,113.040001
2022-08-22,101.550003,109.559998,91.500000,107.190002,23.750000,102.029999,118.709999,55.400002,44.939999,47.299999,46.880001,110.849998,49.730000,106.629997,49.349998,110.089996,82.419998,104.129997,112.660004
2022-08-23,101.510002,109.580002,91.480003,107.209999,23.719999,101.779999,118.620003,55.369999,44.950001,47.340000,46.880001,110.970001,49.759998,106.500000,49.360001,110.099998,82.459999,104.150002,112.040001
2022-08-24,101.239998,109.370003,91.510002,106.830002,23.650000,101.440002,118.379997,55.130001,44.798000,47.279999,46.770000,110.639999,49.730000,106.300003,49.360001,110.089996,82.370003,104.190002,111.220001


In [95]:
#pd.pivot_table(completed_bonds, values='Close', index=completed_bonds.index,columns=['Symbol'])

In [96]:
def return_vwap(data):
    data['VWAP'] = data['Volume'] * data['Adj Close']
    return(data)

In [97]:
prices_df = return_vwap(prices_df)#[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol']]

In [98]:
writer = pd.ExcelWriter('SP1500_screener_multiple.xlsx', engine='xlsxwriter')

data_object = [start,end,prices_df, sp1500_index_df, sp500, sp600, sp400, market_data, completed_fred_pvt, completed_bonds, completed_bonds_pvt, pd.DataFrame(sectors,sector_names), pd.DataFrame(indexes,index_names), screener_sorted, dict_sectors, dict_indexes, dict_sectors_reverse,invert_dict_indexes, list_sector_n_indexes, list_stocks, future_commodities_n_w_index_pvt_w_Fred,price_comm_df_pvt]
               
pickle.dump(data_object, open('..\\data\\interim\\data_object.pkl', 'wb'))
"""
#prices_df.to_excel(writer, sheet_name='SP1500_Sectors_Indexes_Prices')
#prices_df.drop(['Sector', 'Market Cap'], axis=1).to_excel(writer, sheet_name='SP1500Prices')

#sectors_df.to_excel(writer, sheet_name='stock_sectors')

#mktcap_df.to_excel(writer, sheet_name='market_caps')

sp1500_index_df.to_excel(writer, sheet_name='SP1500')

pd.DataFrame(sp500).to_excel(writer, sheet_name='SP500')
pd.DataFrame(sp600).to_excel(writer, sheet_name='SP600')
pd.DataFrame(sp400).to_excel(writer, sheet_name='SP400')
pd.DataFrame(market_data).to_excel(writer, sheet_name='Stock_Fundamentals')
completed_fred_pvt.to_excel(writer, sheet_name='completed_fred_pvt')
completed_bonds.to_excel(writer, sheet_name='completed_bonds')
completed_bonds_pvt.to_excel(writer, sheet_name='completed_bonds_pvt')
pd.DataFrame(pd.DataFrame(sectors,sector_names)).to_excel(writer, sheet_name='vanguard_sectors')
pd.DataFrame(pd.DataFrame(indexes,index_names)).to_excel(writer, sheet_name='spdr_indexes')
screener_metrics.to_excel(writer, sheet_name='Screener')

writer.save()

wb = openpyxl.load_workbook('SP1500_screener_multiple.xlsx')
#openpyxl.Workbook()
new_range = openpyxl.workbook.defined_name.DefinedName('Prices', attr_text='SP1500_Sectors_Indexes_Prices!$A$1:$H$'+str(len(prices_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SP1500', attr_text='SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('Vanguard_Sectors', attr_text='vanguard_sectors!$B$1:$C$'+str(len(sectors)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SPDR_Indexes', attr_text='spdr_indexes!$B$1:$C$'+str(len(indexes)+1))
wb.defined_names.append(new_range)


#wb.defined_names.append(new_range)
wb.save('SP1500_screener_multiple.xlsx')
wb.close()
#named_range = NamedRange("test_nr", [(new_sheet, "A1")])
#wb.add_named_range(new_range)
wb.close()
writer.close()
"""
"""
workbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')
workbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))
workbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))
workbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))
workbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))

"""
#workbook.close()

"\nworkbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')\nworkbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))\nworkbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))\nworkbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))\nworkbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))\n\n"